# ⚠️ Notebook 重构说明（必读）

## 🔴 发现的问题

### 1. **废弃 API 路径** (已在第12单元格修复)
- ❌ 错误：`from quantclassic.data_loader.daily_graph_loader import ...`
- ✅ 正确：`from quantclassic.data_set.graph import DailyBatchDataset, DailyGraphDataLoader`
- 📍 位置：第12单元格 (步骤 3.5-DYNAMIC)

### 2. **列名不一致问题** (需修复)
- ❌ 错误：预处理要求 `industry_name` 和 `ts_code`，但数据只有 `industry_code`
- ✅ 正确：统一使用 `industry_code`，移除 `ts_code`
- 📍 位置：
  - 第6单元格：预处理的 `key_columns`
  - 第10单元格：图构建器的 `industry_col`
  - 第12单元格：`HybridGraphBuilder` 的 `industry_col`
  - 第14单元格：`build_industry_adjacency_matrix` 的 `industry_col`

### 3. **手动实现滚动窗口** (可优化)
- 📌 当前：第12单元格手动实现 `create_daily_loaders_for_window` 函数
- 💡 推荐：使用 `DataManager.create_rolling_daily_loaders()` 内置方法
- 📚 参考：`data_set/manager.py` line 540-650

### 4. **未使用配置驱动链路** (建议优化)
- 📌 当前：直接调用底层 API，代码冗长
- 💡 推荐：使用 YAML 配置 + `TaskRunner`/`qcrun` CLI
- 📚 参考：`config/README.md`, `agent.md`

---

## ✅ 快速修复清单

### 必须修复（否则运行报错）：
1. ✅ **第12单元格**：导入路径已修复为 `quantclassic.data_set.graph`
2. ⚠️ **第6单元格**：修改 `key_columns = ['order_book_id', 'trade_date', 'industry_code']`（移除 `ts_code` 和 `industry_name`）
3. ⚠️ **第10、12、14单元格**：所有 `industry_col='industry_name'` 改为 `'industry_code'`

### 建议优化（提升代码质量）：
4. 💡 **第12单元格**：用 `dm.create_rolling_daily_loaders()` 替代手动实现
5. 💡 **整体架构**：改用 YAML + TaskRunner 配置驱动（参考 config/templates/）

---

## 📋 详细修改指引

### 修复 1：第6单元格 - 预处理列名
```python
# 修改前（错误）
key_columns = ['order_book_id', 'trade_date', 'industry_name', 'ts_code']

# 修改后（正确）
key_columns = ['order_book_id', 'trade_date', 'industry_code']
```

### 修复 2：第10单元格 - 图构建器列名
```python
# 修改前（错误）
industry_builder = IndustryGraphBuilder(
    industry_col='industry_name',  # ❌
    ...
)
hybrid_builder = HybridGraphBuilder(
    industry_col='industry_name',  # ❌
    ...
)

# 修改后（正确）
industry_builder = IndustryGraphBuilder(
    industry_col='industry_code',  # ✅
    ...
)
hybrid_builder = HybridGraphBuilder(
    industry_col='industry_code',  # ✅
    ...
)
```

### 修复 3：第12单元格 - 手动滚动窗口改用内置方法
```python
# 方式1：保留当前手动实现（简单修改）
# 只需修改 HybridGraphBuilder 的 industry_col='industry_code'

# 方式2：使用 DataManager 内置方法（推荐）
# 在第9单元格之后添加：
window_loaders = dm.create_rolling_daily_loaders(
    graph_builder_config={
        'type': 'hybrid',
        'alpha': 0.7,
        'corr_method': 'cosine',
        'top_k': 10,
        'industry_col': 'industry_code',
        'industry_adj_path': 'output/industry_adj_matrix.pt',
        'stock_col': 'order_book_id',
        'add_self_loop': True
    },
    val_ratio=0.2,
    device='cuda' if torch.cuda.is_available() else 'cpu'
)
# 返回 List[DailyLoaderCollection(train, val, test)]
```

### 修复 4：第14单元格 - 构建邻接矩阵列名
```python
# 修改前（错误）
adj_matrix, stock_list, stock_to_idx = build_industry_adjacency_matrix(
    df=df_adj,
    stock_col='order_book_id',
    industry_col='industry_name',  # ❌
    ...
)

# 修改后（正确）
adj_matrix, stock_list, stock_to_idx = build_industry_adjacency_matrix(
    df=df_adj,
    stock_col='order_book_id',
    industry_col='industry_code',  # ✅
    ...
)
```

---

## 🎯 生产环境最佳实践（配置驱动）

参考 `config/templates/lstm_basic.yaml`，创建配置文件：

```yaml
# config/my_experiment.yaml
task:
  name: "lstm_attention_dynamic"
  mode: "train_and_backtest"
  
  dataset:
    class: "DataManager"
    module_path: "quantclassic.data_set"
    config:
      base_dir: "output"
      data_file: "wind_processed_all.parquet"
      stock_col: "order_book_id"
      time_col: "trade_date"
      label_col: "alpha_label"
      feature_cols: ["open", "high", "low", "close", "volume", "amount", "vwap"]
      split_strategy: "rolling"
      rolling_window_size: 2268
      rolling_step: 252
      window_size: 150
      # 图数据配置
      graph_builder_config:
        type: "hybrid"
        alpha: 0.7
        corr_method: "cosine"
        top_k: 10
        industry_col: "industry_code"  # ✅ 正确列名
        industry_adj_path: "output/industry_adj_matrix.pt"
        stock_col: "order_book_id"
        add_self_loop: true

  model:
    class: "HybridGraphModel"
    module_path: "quantclassic.model.hybrid_graph_models"
    config:
      d_feat: 7
      temporal:
        rnn_type: "lstm"
        hidden_size: 32
        num_layers: 2
        use_attention: true
      graph:
        enabled: true
        gat_type: "standard"
        hidden_dim: 32
      fusion:
        hidden_sizes: [128, 64]
        output_dim: 64

  backtest:
    enabled: true
    config:
      n_groups: 10
      rebalance_freq: "biweekly"
      output_dir: "output/backtest_config_driven"
```

**运行方式**：
```bash
# CLI 方式（推荐）
qcrun config/my_experiment.yaml

# SDK 方式
from quantclassic.config import ConfigLoader, TaskRunner
cfg = ConfigLoader.load("config/my_experiment.yaml")
runner = TaskRunner()
results = runner.run(cfg, experiment_name="my_exp")
```

---

## 📚 参考文档
- 配置系统：[config/README.md](../config/README.md)
- 数据管理：[data_set/README.md](../data_set/README.md)
- 模型训练：[model/README.md](../model/README.md)
- AI 协作指南：[agent.md](../agent.md)

# 项目概览与 quantclassic 调用说明

## 1. 场景与目标

- 任务：基于日频 A 股行情和行业信息，构建多因子时序图模型，输出截面因子并做回测。

- 数据范围：2000-2025 年；来源包含 Wind 合并数据 `merged_data.parquet` 与米筐（rqdatac）接口（可选全量抓取）。

- 结果：生成处理后特征/标签、静态行业图与动态图训练结果、多因子回测报告，以及按实验隔离的归档目录。

## 2. 数据处理与预处理

- 数据整理：从 `日度数据与日度特征-中邮/merged_data.parquet` 读取，筛选 2000-2025，字段映射为 `open/high/low/close/vol/amount/vwap`、`trade_date`、`order_book_id`、`industry_code`，保存到 `output/wind_daily_features.parquet`。

- 预处理管道（quantclassic.data_processor）：

  - 标签生成：`y_ret_1d/5d/10d` 简单收益，研报基准使用 `y_ret_10d` 做 SimStock 标签中性化（`alpha_label`）。

  - 清洗：可选去极值、标准化、中性化；缺失值中位数填充；标签在 Dataset 内做窗口排名标准化避免泄露。

  - 输出：`output/caitong_processed_all.parquet`（或 `wind_processed_all.parquet`）。

## 3. 模型与配置（quantclassic.model）

- 数据配置 `DataConfig`：窗口 `window_size=150`，滚动窗口 2268 天、步长 252 天，启用价格对数与量纲归一，支持缓存。

- 模型配置 `CompositeModelConfig`：

  - 时序模块 `TemporalModuleConfig`：LSTM+自注意力，残差与 dropout。

  - 图模块 `GraphModuleConfig`：GAT，top_k=10，支持静态行业图或动态图。

  - 融合模块 `FusionModuleConfig`：两层 MLP，`output_dim=N_FACTORS=64` 生成多因子截面矩阵。

  - 训练：`n_epochs=20`，`batch_size=1000`，Adam，Plateau 调度器，可选相关性正则化 `lambda_corr`。

- 静态行业邻接矩阵：`build_industry_adjacency_matrix` 基于行业分类一次构建，保存 `output/industry_adj_matrix.pt`，避免使用目标列导致泄露。

## 4. 数据加载与训练模式

- 静态批次模式（RollingWindowTrainer）：固定 batch，静态行业图；可检测并复用已保存的窗口模型 (`output/rolling_models/window_*_model.pth`)。

- 动态批次模式（DailyBatchDataset + DailyGraphDataLoader + DynamicGraphTrainer）：

  - 按交易日构建 batch，batch_size=当日股票数。

  - 图构建器：`IndustryGraphBuilder` (静态)、`CorrGraphBuilder` (相关性)、`HybridGraphBuilder` (加权融合)，每日动态生成邻接矩阵。

  - 支持滚动窗口全流程训练、IC 评估、预测合并以及多因子输出。

## 5. 回测与评估（quantclassic.backtest）

- 多因子回测：`MultiFactorBacktest`，IC 加权集成，分组数=10，双周调仓，支持成本、图表和 Excel 报告；单因子可选 `FactorBacktestSystem`。

- 关键输入：

  - 因子列 `pred_factor_*`（或单列 `factor_raw_std`），收益列优先 `future_return`，索引列 `order_book_id/trade_date`。

  - 回测输出目录：`output/backtest_multifactor`（静态）或 `output/backtest_dynamic_graph`（动态图）。

- 评估指标：截面 IC/ICIR、IC 胜率；组合年化收益、波动、夏普、回撤等，并支持多集成方法对比（mean / ic_weighted / best）。

## 6. 实验与复现（quantclassic.workflow.R）

- 运行 `步骤 12` 可一键归档：配置、预测结果、组合、指标、模型、图表与特征列表，存放于 `output/experiments/<timestamp>/`。

- 生成 `EXPERIMENT_REPORT.txt`，可直接加载 parquet/配置文件复现或分析。

## 7. 使用提示

- 先跑数据处理与预处理，再选择静态或动态训练路径，最后回测。

- 若遇到配额或字体问题，按首个单元格提示安装字体或检查 rqdatac 配额。

- 训练/回测前确认 GPU 可用性与路径（output/ 目录）存在，避免缺文件错误。


In [2]:
import sys
import warnings
warnings.filterwarnings('ignore')

# 设置matplotlib中文显示 - Linux/macOS适配
import matplotlib
import matplotlib.pyplot as plt
import platform


# 中文字体配置（需要系统已安装中文字体）
# Linux: sudo apt-get install fonts-wqy-microhei
# macOS: 已内置中文字体
# Windows: 已内置中文字体
os_name = platform.system()
if os_name == 'Darwin':  # macOS
    matplotlib.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'PingFang SC', 'STHeiti']
    print("  6. 中文字体: Arial Unicode MS (macOS)")
elif os_name == 'Linux':
    # 尝试设置中文字体，如果没有则使用英文
    try:
        matplotlib.rcParams['font.sans-serif'] = ['WenQuanYi Micro Hei', 'Noto Sans CJK SC', 'DejaVu Sans']
        print("  6. 中文字体: WenQuanYi Micro Hei (Linux)")
        print("     若中文显示异常，请运行: sudo apt-get install fonts-wqy-microhei")
    except:
        matplotlib.rcParams['font.sans-serif'] = ['DejaVu Sans']
        print("  6. ⚠️  未找到中文字体，使用英文标签")
        print("     安装中文字体: sudo apt-get install fonts-wqy-microhei")
else:  # Windows
    matplotlib.rcParams['font.sans-serif'] = ['Microsoft YaHei', 'SimHei']
    print("  6. 中文字体: Microsoft YaHei (Windows)")

matplotlib.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
print("=" * 80)

  6. 中文字体: WenQuanYi Micro Hei (Linux)
     若中文显示异常，请运行: sudo apt-get install fonts-wqy-microhei


In [3]:
import rqdatac
import os

# 在RiceQuant官网获取您的API Token
# 建议：将账号密码设置为环境变量 RQ_USERNAME 和 RQ_PASSWORD
try:
    username = os.environ.get('RQ_USERNAME', '15002430024')
    password = os.environ.get('RQ_PASSWORD', '280379sysB#')

    rqdatac.init(username, password)    print(f"初始化失败: {e}")

    print("RiceQuant 初始化成功！")except Exception as e:

SyntaxError: invalid syntax (1954241468.py, line 10)

In [ ]:
# # =============================================================================
# # 步骤 2A: 数据提取与预处理配置（可选 - 从零开始处理）
# # =============================================================================
# # 如果需要重新提取和处理数据，运行此cell
# # 否则直接运行下一个cell使用已处理的数据


# import sys
# sys.path.insert(0, '/home/u2025210237/jupyterlab')

# # 🔴 强制重新加载所有 quantclassic 模块（确保使用最新代码）
# import importlib
# modules_to_reload = [key for key in list(sys.modules.keys()) if 'quantclassic' in key]
# for mod in modules_to_reload:
#     del sys.modules[mod]

# from pathlib import Path
# from quantclassic.data_fetch.config_manager import (
#     TimeConfig, DataSourceConfig, UniverseConfig, FeatureConfig,
#     DataFieldsConfig, StorageConfig, ProcessConfig
# )
# from quantclassic.data_processor.preprocess_config import (
#     PreprocessConfig, ProcessMethod, PreprocessTemplates
# )

# print("\n" + "=" * 80)
# print("📥 数据提取与预处理配置")
# print("=" * 80)

# # ==================== 1. 数据提取配置 ====================
# print("\n【1. 数据提取配置】")

# # 1.1 时间配置
# time_config = TimeConfig(
#     start_date='2010-01-01',    # 起始日期
#     end_date='2024-12-31',      # 结束日期
#     frequency='1d'              # 数据频率：1d-日频
# )
# print(f"✅ 时间范围: {time_config.start_date} ~ {time_config.end_date}")

# # 1.2 数据源配置
# data_source_config = DataSourceConfig(
#     source='rq',                # 数据源：rq-米筐
#     market='cn',                # 市场：cn-中国A股
#     auth_method='account',     # 认证方式：账号+密码
#     username='15725261010',    # 米筐账号
#     password='zpy105827317'    # 米筐密码
# )
# print(f"✅ 数据源: {data_source_config.source} (认证: {data_source_config.auth_method})")

# # 1.3 股票池配置
# universe_config = UniverseConfig(
#     universe_type='all_a',     # 股票池：all_a-全部A股
#     exclude_st=True,            # 排除ST股票
#     main_board_only=False,      # 不限制主板
#     min_list_days=0            # 最小上市天数
# )
# print(f"✅ 股票池: {universe_config.universe_type}")

# # 1.4 数据字段配置（仅保留 7 个行情字段，关闭估值/股本提取）
# # 🔴 注意: amount 会自动映射为 total_turnover，vwap 会单独获取
# data_fields_config = DataFieldsConfig(
#     price_fields=[              # 行情数据字段
#         'open', 'high', 'low', 'close', 'volume', 'amount', 'vwap'
#     ],
#     include_vwap=True,          # 🔴 修改: 启用 VWAP 单独获取
#     valuation_fields=[],        # 关闭估值字段
#     share_fields=[],            # 关闭股本字段
#     industry_source='sws',      # 行业分类：sws-申万
#     industry_level=1            # 行业级别
# )
# print(f"✅ 字段数量: 行情{len(data_fields_config.price_fields)}个，估值已关闭")

# # 1.5 存储配置
# storage_config = StorageConfig(
#     save_dir='rq_data_parquet_all',     # 保存目录
#     file_format='parquet',          # 文件格式
#     compression=None                # 压缩方式
# )
# storage_config.ensure_dirs()  # 🆕 确保目录存在
# resume_state_path = Path(storage_config.save_dir) / '.resume_state.json'
# print(f"✅ 存储目录: {storage_config.save_dir}")

# # 1.6 处理配置（包含断点续传）
# process_config = ProcessConfig(
#     batch_size=500,
#     retry_times=3,
#     sleep_interval=0.1,
#     resume_enabled=True,
#     resume_entities=['price', 'valuation', 'share']
# )
# if process_config.resume_enabled:
#     print(f"✅ 断点续传: 已开启 (缓存文件: {resume_state_path})")

# # ==================== 2. 执行数据提取 ====================
# print("\n【2. 数据提取执行】")
# print("⚠️  开始提取数据...")

# try:
#     # 导入数据管道
#     from quantclassic.data_fetch.pipeline import DataPipeline
    
#     # 创建数据管道
#     pipeline = DataPipeline(
#         time_config=time_config,
#         source_config=data_source_config,
#         universe_config=universe_config,
#         fields_config=data_fields_config,
#         storage_config=storage_config,
#         process_config=process_config,
#     )
    
#     if resume_state_path.exists():
#         print(f"  🔄 检测到已有断点缓存: {resume_state_path}")
#     else:
#         print("  🔄 首次运行，将创建断点缓存 ...")
    
#     # 提取数据
#     print("  正在提取数据...")
#     df = pipeline.run_full_pipeline()
    
#     # 🔴 检查返回值是否有效
#     if df is None:
#         raise ValueError(
#             "Pipeline 返回 None！请检查上方日志中的错误信息。\n"
#             "常见原因：\n"
#             "  1. 米筐 API 配额超限 (Quota exceeded)\n"
#             "  2. 字段名不被支持 (amount 应自动映射为 total_turnover)\n"
#             "  3. 网络连接问题"
#         )
    
#     if df.empty:
#         raise ValueError("Pipeline 返回空 DataFrame！请检查数据源和时间范围配置。")
    
#     if resume_state_path.exists():
#         print(f"  ✅ 断点信息已更新: {resume_state_path}")
    
#     # 🆕 保存提取的数据
#     import os
#     os.makedirs('output', exist_ok=True)
#     save_path = 'output/wind_processed_all.parquet'
#     df.to_parquet(save_path)
#     print(f"  ✅ 数据已保存到: {save_path}")
    
#     # 同时赋值给 df_raw 供后续单元格使用
#     df_raw = df
    
#     print(f"\n✅ 数据提取完成!")
#     print(f"  数据形状: {df.shape}")
#     print(f"  时间范围: {df['trade_date'].min()} ~ {df['trade_date'].max()}")
#     print(f"  股票数量: {df['order_book_id'].nunique()}")
    
#     # 显示数据列信息
#     print(f"\n📊 数据列信息（共{len(df.columns)}列）：")
#     print(f"  前20列: {list(df.columns[:20])}")
#     if len(df.columns) > 20:
#         print(f"  ... 还有 {len(df.columns) - 20} 列")
    
#     # 显示数据样本
#     print(f"\n📋 数据样本（前5行）：")
#     print(df.head().to_string())
    
# except Exception as e:
#     print(f"\n❌ 数据提取失败: {e}")
#     print("\n💡 排查建议:")
#     print("   1. 检查上方日志中的具体错误信息")
#     print("   2. 如果是 'Quota exceeded'，请等待一段时间后重试")
#     print("   3. 如果是字段错误，请确认已重启内核加载最新代码")
#     print("   4. 运行: import rqdatac; print(rqdatac.user.get_quota()) 查看配额")
#     print(f"   5. 如需完全重新提取，请删除缓存文件: {resume_state_path}")
#     raise

/home/u2025210237/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-12-03 16:29:03 - quantclassic.data_fetch.pipeline - INFO - 初始化数据流水线: ConfigManager(source=rq, universe=all_a, period=2010-01-01~2024-12-31)



📥 数据提取与预处理配置

【1. 数据提取配置】
✅ 时间范围: 2010-01-01 ~ 2024-12-31
✅ 数据源: rq (认证: account)
✅ 股票池: all_a
✅ 字段数量: 行情7个，估值已关闭
✅ 存储目录: rq_data_parquet_all
✅ 断点续传: 已开启 (缓存文件: rq_data_parquet_all/.resume_state.json)

【2. 数据提取执行】
⚠️  开始提取数据...


2025-12-03 16:29:03 - quantclassic.data_fetch.data_fetcher - INFO - 米筐 API 初始化成功!
2025-12-03 16:29:03 - quantclassic.data_fetch.data_fetcher - INFO - API信息: None
2025-12-03 16:29:03 - quantclassic.data_fetch.pipeline - INFO - 
2025-12-03 16:29:03 - quantclassic.data_fetch.pipeline - INFO - 开始执行完整数据流水线
2025-12-03 16:29:03 - quantclassic.data_fetch.pipeline - INFO - ================================================================================
2025-12-03 16:29:03 - quantclassic.data_fetch.pipeline - INFO - 配置: ConfigManager(source=rq, universe=all_a, period=2010-01-01~2024-12-31)
2025-12-03 16:29:03 - quantclassic.data_fetch.pipeline - INFO - 保存中间结果: True
2025-12-03 16:29:03 - quantclassic.data_fetch.pipeline - INFO - 执行验证: True
2025-12-03 16:29:03 - quantclassic.data_fetch.pipeline - INFO - ================================================================================

2025-12-03 16:29:03 - quantclassic.data_fetch.pipeline - INFO - 
2025-12-03 16:29:03 - quantclassic.data_fetch.pipe

rqdatac version: 3.3.1
server address: rqdatad-pro.ricequant.com:16011
You are using account: 15725261010
  🔄 首次运行，将创建断点缓存 ...
  正在提取数据...


2025-12-03 16:29:03 - quantclassic.data_fetch.data_fetcher - INFO - 获取到 5122 只A股股票
2025-12-03 16:29:03 - quantclassic.data_fetch.data_fetcher - INFO - 排除ST股票: 5122 -> 4936
2025-12-03 16:29:03 - quantclassic.data_fetch.data_fetcher - INFO - 排除ST股票: 5122 -> 4936
/home/u2025210237/jupyterlab/quantclassic/data_fetch/data_fetcher.py:188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stocks['listed_date'] = pd.to_datetime(df_stocks['listed_date'])
2025-12-03 16:29:03 - quantclassic.data_fetch.data_fetcher - INFO - 最终股票池: 4936 只股票
2025-12-03 16:29:03 - quantclassic.data_fetch.data_fetcher - INFO - 股票市场分布:
order_book_id
.XSHE    2725
.XSHG    2211
Name: count, dtype: int64
2025-12-03 16:29:03 - quantclassic.data_fetch.data_fetcher - INFO - === 获取交易日历 

  ✅ 断点信息已更新: rq_data_parquet_all/.resume_state.json
  ✅ 数据已保存到: output/caitong_raw_all.parquet

✅ 数据提取完成!
  数据形状: (5836140, 48)
  时间范围: 2010-01-04 00:00:00 ~ 2024-12-31 00:00:00
  ✅ 数据已保存到: output/caitong_raw_all.parquet

✅ 数据提取完成!
  数据形状: (5836140, 48)
  时间范围: 2010-01-04 00:00:00 ~ 2024-12-31 00:00:00
  股票数量: 2000

📊 数据列信息（共48列）：
  前20列: ['trade_date', 'order_book_id', 'open', 'high', 'low', 'close', 'vol', 'amount', 'vwap', 'total_share', 'industry_code', 'industry_name', 'pct_chg', 'pre_close', 'amplitude', 'turnover_rate', 'volume_ratio', 'ret_1d', 'ret_5d', 'ret_10d']
  ... 还有 28 列

📋 数据样本（前5行）：
  trade_date order_book_id        open        high         low       close           vol        amount       vwap  total_share industry_code industry_name   pct_chg   pre_close  amplitude  turnover_rate  volume_ratio    ret_1d  ret_5d  ret_10d  ret_20d  vol_20d  ma_close_5d     ma_vol_5d  ma_close_10d  ma_vol_10d  ma_close_20d  ma_vol_20d  ma_close_60d  ma_vol_60d  close_lag_1  close_lag_2

In [ ]:
# =============================================================================
# 步骤 1: Wind日度数据加载与字段转换
# 数据范围：2000年-2025年
# 字段：时间、股票代码、7个特征、中信二级行业
# =============================================================================

import pandas as pd
import numpy as np
from pathlib import Path

print("\n" + "=" * 80)
print("📊 步骤 2: Wind日度数据加载与字段转换")
print("=" * 80)

# ==================== 0. 加载合并后的数据 ====================
print("\n【0. 数据加载】")
data_path = '/home/u2025210237/jupyterlab/日度数据与日度特征-中邮/merged_data.parquet'
df_raw = pd.read_parquet(data_path)
print(f"  原始数据: {df_raw.shape[0]:,} 行, {df_raw.shape[1]} 列")

# ==================== 1. 时间筛选：2000-2025年 ====================
print("\n【1. 时间筛选：2000-2025年】")
df_raw['TRADE_DT'] = df_raw['TRADE_DT'].astype(str)
df = df_raw[(df_raw['TRADE_DT'] >= '20000101') & (df_raw['TRADE_DT'] <= '20251231')].copy()
print(f"  筛选后: {len(df):,} 行")
print(f"  日期范围: {df['TRADE_DT'].min()} ~ {df['TRADE_DT'].max()}")

# ==================== 2. 字段选择与重命名 ====================
print("\n【2. 字段选择与重命名】")

# 定义字段映射（Wind字段 → 模型字段）
column_mapping = {
    # 关键标识
    'TRADE_DT': 'trade_date',           # 交易日期
    'S_INFO_WINDCODE': 'order_book_id', # 股票代码
    
    # 7个特征
    'S_DQ_ADJCLOSE': 'close',           # 复权收盘价（用于计算收益率）
    'S_DQ_ADJOPEN': 'open',             # 复权开盘价
    'S_DQ_ADJHIGH': 'high',             # 复权最高价
    'S_DQ_ADJLOW': 'low',               # 复权最低价
    'S_DQ_VOLUME': 'vol',            # 成交量
    'S_DQ_AMOUNT': 'amount',            # 成交额
    'S_DQ_AVGPRICE': 'vwap',            # 均价(VWAP)
    
    # 行业分类
    'IND2': 'industry_code',            # 中信二级行业代码
}

# 选择需要的列
selected_cols = list(column_mapping.keys())
missing_cols = [c for c in selected_cols if c not in df.columns]
if missing_cols:
    print(f"  ⚠️ 缺失列: {missing_cols}")
    selected_cols = [c for c in selected_cols if c in df.columns]

df = df[selected_cols].copy()

# 重命名
df = df.rename(columns=column_mapping)
print(f"  选择字段: {list(df.columns)}")

# ==================== 3. 数据类型转换 ====================
print("\n【3. 数据类型转换】")

# 日期转换为datetime
df['trade_date'] = pd.to_datetime(df['trade_date'], format='%Y%m%d')
print(f"  trade_date → datetime: {df['trade_date'].dtype}")

# 数值列转换
numeric_cols = ['close', 'open', 'high', 'low', 'vol', 'amount', 'vwap']
for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# ==================== 4. 数据质量检查 ====================
print("\n【4. 数据质量检查】")
print(f"  总行数: {len(df):,}")
print(f"  股票数: {df['order_book_id'].nunique():,}")
print(f"  交易日数: {df['trade_date'].nunique():,}")
print(f"  行业数: {df['industry_code'].nunique()}")

# 缺失值统计
print("\n  缺失值统计:")
null_stats = df.isnull().sum()
for col in df.columns:
    null_count = null_stats[col]
    null_pct = null_count / len(df) * 100
    if null_pct > 0:
        print(f"    {col}: {null_count:,} ({null_pct:.2f}%)")

# ==================== 5. 剔除缺失行业的数据 ====================
print("\n【5. 剔除缺失行业的数据】")
before_len = len(df)
df = df[df['industry_code'].notna()].copy()
print(f"  剔除无行业数据: {before_len - len(df):,} 行")
print(f"  剩余: {len(df):,} 行")

# ==================== 6. 保存处理后数据 ====================
print("\n【6. 数据预览与保存】")
print(df.head(10).to_string())

# 保存
output_dir = Path('output')
output_dir.mkdir(exist_ok=True)
output_path = output_dir / 'wind_daily_features.parquet'
df.to_parquet(output_path, index=False)
print(f"\n✅ 数据已保存: {output_path}")

# 兼容后续单元格
df_raw = df.copy()
print(f"\n📊 最终数据: {df.shape[0]:,} 行, {df.shape[1]} 列")
print(f"   字段: {list(df.columns)}")


📊 步骤 2: Wind日度数据加载与字段转换

【0. 数据加载】
  原始数据: 16,189,236 行, 36 列

【1. 时间筛选：2000-2025年】
  筛选后: 15,384,936 行
  日期范围: 20000104 ~ 20251226

【2. 字段选择与重命名】
  选择字段: ['trade_date', 'order_book_id', 'close', 'open', 'high', 'low', 'vol', 'amount', 'vwap', 'industry_code']

【3. 数据类型转换】
  trade_date → datetime: datetime64[ns]

【4. 数据质量检查】
  总行数: 15,384,936
  股票数: 5,760
  交易日数: 6,298
  行业数: 108

  缺失值统计:
    industry_code: 775,837 (5.04%)

【5. 剔除缺失行业的数据】
  剔除无行业数据: 775,837 行
  剩余: 14,609,099 行

【6. 数据预览与保存】
  trade_date order_book_id    close     open     high      low        vol      amount     vwap industry_code
0 2016-06-15     600227.SH  33.5800  32.8300  34.0400  32.6000  101828.63  59317.6920   5.8252        b10601
1 2014-05-28     002262.SZ  70.1700  68.2400  70.5400  67.5700   17529.77  36298.9779  20.7070        b10i01
2 2025-11-05     920765.BJ  36.1000  36.4155  36.4155  35.3972   13295.45  33419.1380  25.1358        b10903
3 2004-05-10     600863.SH  23.5400  23.5800  23.9900  23.3800   

In [ ]:
# =============================================================================
# 步骤 2: 基础数据预处理管道（不含窗口变换）
# 流程：标签生成 → 去极值 → 标准化 → 特征中性化 → 标签中性化 → 缺失值填充
# =============================================================================

import importlib
import sys
import numpy as np
import pandas as pd
from pathlib import Path
import sys
sys.path.insert(0, '/home/u2025210237/jupyterlab')
from quantclassic.data_processor import (
    ProcessMethod, 
    PreprocessConfig, 
    DataPreprocessor
)



# ==================== 0. 数据加载与特征筛选 ====================
print("\n【0. 数据加载与特征筛选】")

if 'df_raw' not in locals() or df_raw is None:
    try:
        df_raw = pd.read_parquet('output/wind_daily_features.parquet')
        print(f"  ✅ 成功加载数据: {df_raw.shape}")
    except:
        raise ValueError("需要先运行数据提取cell")

# 保留关键列 + 数值列（industry_code用于后续动态图构建）
key_columns = ['order_book_id', 'trade_date', 'industry_code']
numeric_columns = df_raw.select_dtypes(include=[np.number]).columns.tolist()
columns_to_keep = list(set(key_columns) & set(df_raw.columns)) + numeric_columns
columns_to_keep = list(dict.fromkeys(columns_to_keep))

df_input = df_raw[columns_to_keep].copy()
print(f"  筛选后形状: {df_input.shape}")

# ==================== 1. 创建预处理配置 ====================
print("\n【1. 创建预处理配置】")
config = PreprocessConfig()

# ==================== 2. 配置处理流程（研报标准6步） ====================
print("\n【2. 配置处理流程（6步）】")

# 步骤1: 生成标签
# 所有参数直接在 add_step 中配置，无需单独设置 config.label_config
print("  1️⃣  生成多周期标签: y_ret_1d/5d/10d")
config.add_step(
    name='生成多周期标签', 
    method=ProcessMethod.GENERATE_LABELS,
    # === LabelGeneratorConfig 参数 ===
    stock_col='order_book_id',   # 股票代码列
    time_col='trade_date',       # 时间列
    price_col='close',           # 价格列（分子：未来价格）
    base_price_col='close',      # 基准价格列（分母）: 'close'=T+1基准(研报), None=T日基准(传统)
    label_type='return',         # 标签类型: 'return' | 'class'
    return_periods=[1, 5, 10],   # 收益率周期列表（交易日）
    return_method='simple',      # 收益率计算: 'simple' | 'log'
    label_prefix='y_ret'         # 标签前缀 → y_ret_1d, y_ret_5d, y_ret_10d
)

# 步骤5: 标签SimStock中性化
# 所有参数直接在 add_step 中配置，无需单独设置 config.neutralize_config
print("  5️⃣  标签SimStock中性化 (Lookback=252)")
config.add_step(
    name='标签SimStock中性化', 
    method=ProcessMethod.SIMSTOCK_LABEL_NEUTRALIZE,
    # === NeutralizeConfig 中的 SimStock 参数 ===
    label_column='y_ret_10d',      # 输入标签列（用于计算相似度）
    output_column='alpha_label',  # 输出列名
    similarity_threshold=0.7,     # 相似度阈值: [0.5宽松, 0.7平衡, 0.8严格]
    lookback_window=252,          # 回看窗口（交易日）: 60=3月, 252=1年
    min_similar_stocks=5,         # 最少相似股票数
    recalc_interval=20,           # 相关性矩阵重算间隔: 1=每天, 20=每月(推荐)
    correlation_method='pearson'  # 相关性方法: 'pearson' | 'spearman'
)

# 🔴 注意：标签的排名标准化已移至 Dataset 的 __getitem__ 中
# 在时序窗口内做 Rank 标准化，避免未来信息泄露

# 步骤6: 缺失值填充
print("  6️⃣  缺失值填充: 中位数")
config.add_step(
    name='全局缺失值填充', 
    method=ProcessMethod.FILLNA_MEDIAN
)

# ==================== 3. 执行预处理 ====================
print("\n【3. 执行预处理管道】")
processor = DataPreprocessor(config)

df_processed = processor.fit_transform(
    df=df_input,
    target_column='y_ret_10d'  # SimStock需要此参数计算相关性
)

# ==================== 4. 结果检查与保存 ====================
print("\n【4. 结果检查】")
print(f"  处理后形状: {df_processed.shape}")

# 检查标签列
label_cols = [c for c in df_processed.columns if c.startswith('y_ret')]
print(f"  生成的标签列: {label_cols}")

# 检查SimStock标签
if 'alpha_label' in df_processed.columns:
    print(f"\n  ✅ SimStock标签 (alpha_label) 生成成功")
    print(f"     均值: {df_processed['alpha_label'].mean():.6f}")
    print(f"     标准差: {df_processed['alpha_label'].std():.6f}")
else:
    print("\n  ⚠️ SimStock标签未生成，请检查日志")

# 保存
config_dir = Path('output/configs')
config_dir.mkdir(parents=True, exist_ok=True)
config.to_yaml('output/configs/preprocess_config.yaml')
df_processed.to_parquet('output/wind_processed_all.parquet', index=False)
print(f"\n✅ 数据已保存: output/wind_processed_all.parquet")

# 兼容性赋值
df_raw1 = df_processed.copy()

/home/u2025210237/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



【0. 数据加载与特征筛选】
  ✅ 成功加载数据: (14609099, 10)
  筛选后形状: (14609099, 10)

【1. 创建预处理配置】

【2. 配置处理流程（6步）】
  1️⃣  生成多周期标签: y_ret_1d/5d/10d
  5️⃣  标签SimStock中性化 (Lookback=252)
  6️⃣  缺失值填充: 中位数

【3. 执行预处理管道】


处理: 标签SimStock中性化:  33%|███▎      | 1/3 [00:12<00:24, 12.09s/步骤]


🚀 SimStock 标签中性化（Numpy加速 + 降频更新版）
📊 步骤1/4: 转换为宽表矩阵 (Pivot)...
  原始数据: 14,547,107 行
  宽表维度: 6,199 天 × 5,632 只股票
  回溯窗口: 252 天
  相关性阈值: 0.70
  更新频率: 每 20 天重新计算一次相似性（关键优化！）
  预计重算次数: 298 次（vs 原方案 5947 次）

📈 步骤2/4: 滚动计算 Alpha (5,947 个有效日期)...


计算Alpha: 100%|██████████| 5947/5947 [13:31<00:00,  7.33天/s]
/home/u2025210237/jupyterlab/quantclassic/data_processor/feature_processor.py:726: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  alpha_series = alpha_df.stack(dropna=False)



📦 步骤3/4: 堆叠结果回长表 (Stack)...
🔗 步骤4/4: 合并回原始数据...

✅ SimStock 中性化完成!
  输出列: alpha_label
  有效样本: 14,546,650 / 14,547,107 (100.0%)
  Alpha均值: 0.000200


处理: 全局缺失值填充:  67%|██████▋   | 2/3 [14:05<08:15, 495.13s/步骤]    骤]

  Alpha标准差: 0.032240



处理: 全局缺失值填充: 100%|██████████| 3/3 [14:14<00:00, 284.85s/步骤]



【4. 结果检查】
  处理后形状: (14547107, 14)
  生成的标签列: ['y_ret_1d', 'y_ret_5d', 'y_ret_10d']

  ✅ SimStock标签 (alpha_label) 生成成功
     均值: 0.000200
     标准差: 0.032240

✅ 数据已保存: output/wind_processed_all.parquet


In [ ]:
# df_pro = pd.read_parquet('/home/u2025210237/jupyterlab/output/caitong_processed_all.parquet')
df_raw1.to_parquet('/home/u2025210237/jupyterlab/output/wind_processed_all.parquet', index=False)

In [1]:
# =============================================================================
# 步骤 1: 创建OOP配置 (使用新的配置系统)
# =============================================================================
# 🔴 研报 Baseline 修改：
#   1. output_dim = 8 (多因子输出 N×F)
#   2. 使用静态行业邻接矩阵，而非动态相关性图
#   3. 训练后对 F 个因子做平均或集成
# =============================================================================

import sys
sys.path.insert(0, '/home/u2025210237/jupyterlab')

# 🔴 强制删除并重新加载模块（确保使用最新代码）
import importlib
modules_to_reload = [
    'quantclassic.model.modular_config',
    'quantclassic.model.base_model',
    'quantclassic.model.hybrid_graph_models',
    'quantclassic.model.build_industry_adj',
    'quantclassic.data_manager.rolling_trainer',
    'quantclassic.data_manager.manager',
    'quantclassic.data_manager'
]

# 先删除缓存的模块
for mod in modules_to_reload:
    if mod in sys.modules:
        del sys.modules[mod]

# 重新导入
from pathlib import Path
from quantclassic.model.model_config import GRUConfig, LSTMConfig
from quantclassic.data_set.config import DataConfig
from quantclassic.backtest.backtest_config import BacktestConfig
from quantclassic.model.modular_config import (
    TemporalModuleConfig,
    GraphModuleConfig,
    FusionModuleConfig,
    CompositeModelConfig,
    ModelConfigBuilder,
    ConfigTemplates
)

print("\n" + "=" * 80)
print("📝 步骤 1: 创建配置对象 (研报 Baseline 版本)")
print("=" * 80)

# =============================================================================
# 🆕 研报 Baseline 关键参数
# =============================================================================
N_FACTORS = 64  # 🔴 多因子输出维度 F（研报建议 4~16）
USE_STATIC_ADJ = False  # 🔴 不使用静态行业邻接矩阵

print(f"\n📌 研报 Baseline 配置:")
print(f"   多因子输出维度 F = {N_FACTORS}")
print(f"   使用静态邻接矩阵 = {USE_STATIC_ADJ}")

# 1. 数据配置
print("\n1️⃣ 数据配置 (DataConfig)")
data_config = DataConfig(
    base_dir='output',
    data_file='wind_processed_all.parquet',
    stock_col='order_book_id',
    time_col='trade_date',
    label_col='alpha_label',
    exclude_cols=['y_ret_1d','y_ret_5d','y_ret_10d'],
    feature_cols=['open','high','low','close','vol','amount','vwap'],  # 自动检测
    split_strategy='rolling',
    # ========== 滚动窗口配置（7年训练+2年验证+1年测试）==========
    rolling_window_size=2268,  # 9年（7年训练+2年验证）= 252*9
    rolling_step=252,          # 1年测试，然后滚动到下一年
    # ==========================================================
    window_size=150,
    batch_size=1000,   # 🔴 修改: 设置为1000 (中证800约800只，留点余量防止溢出拆分)
    num_workers=4,
    train_ratio=0.7,   # 注：滚动窗口模式下，验证集比例在训练时通过 val_ratio 参数设置
    val_ratio=0.2,
    test_ratio=0.1,
    enable_window_transform=True,
    window_price_log=True,
    price_cols=['open', 'high', 'low', 'close', 'vwap'],
    close_col='close',
    window_volume_norm=True,
    volume_cols=['vol', 'amount'],
    # 🆕 标签窗口级时序排名标准化（避免未来信息泄露）
    label_rank_normalize=True,
    label_rank_output_range=(-1, 1),
    enable_cache=True,
    cache_dir='cache/data_manager'
)

print(f"✅ 数据配置创建成功")
print(f"  数据文件: {data_config.data_file}")
print(f"  窗口大小: {data_config.window_size}")
print(f"  批次大小: {data_config.batch_size} (全市场模式)")
print(f"  划分策略: {data_config.split_strategy}")
print(f"  划分比例: {data_config.train_ratio}/{data_config.val_ratio}/{data_config.test_ratio}")

# =============================================================================
# 2. 混合模型配置 (🆕 研报 Baseline: 多因子输出)
# =============================================================================
# 🔴 研报 Baseline 关键改动：
#   1. output_dim = N_FACTORS (输出 N×F 因子矩阵)
#   2. 使用静态行业邻接矩阵
#   3. 后续对 F 个因子做平均或集成

# 步骤 1: 独立配置每个模块
temporal_config = TemporalModuleConfig(
    rnn_type='lstm',
    hidden_size=32,           # 🔧 研报建议: 64~128
    num_layers=2,
    use_attention=True,
    attention_type='self',
    dropout=0.2,
    use_residual=True         # 🆕 启用残差连接 (LayerNorm + Skip Connection)
)

graph_config = GraphModuleConfig(
    enabled=True,             # 启用 GAT
    gat_type='standard',      # 🔴 使用标准 GAT (基于行业邻接矩阵)
    hidden_dim=32,            # 🔧 研报建议: 32~64
    heads=4,
    top_k_neighbors=10,
    dropout=0.2,
    use_residual=True         # 🆕 启用残差连接 (LayerNorm + Skip Connection)
)

# 🔴 研报 Baseline: 多因子输出
fusion_config = FusionModuleConfig(
    hidden_sizes=[128, 64],        # 🔧 研报建议: [64] 或 [128, 64]
    activation='relu',
    dropout=0.2,
    output_dim=N_FACTORS      # 🔴 关键修改: 输出 F 个因子 (N×F 矩阵)
)

# 步骤 2: 组合成完整模型配置
model_config = CompositeModelConfig(
    temporal=temporal_config,
    graph=graph_config,
    fusion=fusion_config,
    d_feat=7,
    n_epochs=20,              # 增加训练轮数
    early_stop=6,             # 增加早停耐心
    batch_size=1000,          # 🔴 修改: 同步设置为1000
    learning_rate=0.001,      # 降低学习率 (0.01 -> 0.001)
    optimizer='adam',
    # 🔴 研报 Baseline: 使用静态行业邻接矩阵
    adj_matrix_path='output/industry_adj_matrix.pt' if USE_STATIC_ADJ else None,
    # ========== 🆕 学习率自动调整 (ReduceLROnPlateau) ==========
    use_scheduler=True,        # 启用学习率调度器
    scheduler_type='plateau',  # 当验证损失停止下降时自动降低学习率
    scheduler_patience=2,      # 连续3个epoch无改善后降低学习率
    scheduler_factor=0.5,      # 每次降为原来的0.5倍
    scheduler_min_lr=1e-6,     # 最小学习率下限
    # ========== 🆕 残差连接 ==========
    use_residual=True,         # 全局残差连接开关
    # ========== 🆕 Graph-Aware Inference ==========
    graph_inference_mode='batch',  # 🔴 使用 batch 模式（静态图更稳定）
    max_neighbors=10,                         # 最大邻居数
    stock_idx_position=2,                     # 明确指定股票索引位置 (X, y, stock_idx)
    # ========== 🆕 相关性正则化 (Correlation Regularization) ==========
    lambda_corr=0.01,          # 相关性正则化权重 (0=禁用, 推荐0.001~0.1)
)

# 步骤 3: 验证并查看摘要
model_config.validate()
print(model_config.summary())
print(model_config)

# 4. 回测配置
print("\n4️⃣ 回测配置 (BacktestConfig)")
backtest_config = BacktestConfig(
    window_size=150,
    batch_size=1000, # 🔴 修改: 回测也使用1000
    device='cuda',
    n_groups=10
)

print(f"✅ 回测配置创建成功")
print(f"  分组数量: {backtest_config.n_groups}")

# 5. 保存配置
print("\n5️⃣ 保存配置文件")
config_dir = Path('output/configs')
config_dir.mkdir(parents=True, exist_ok=True)

data_config.to_yaml('output/configs/data_config.yaml')
temporal_config.to_yaml('output/configs/temporal_config.yaml')
graph_config.to_yaml('output/configs/graph_config.yaml')
fusion_config.to_yaml('output/configs/fusion_config.yaml')
model_config.to_yaml('output/configs/model_config.yaml')

print(f"✅ 配置已保存")
print(f"  数据配置: output/configs/data_config.yaml")
print(f"  时间序列配置: output/configs/temporal_config.yaml")
print(f"  图配置: output/configs/graph_config.yaml")
print(f"  融合配置: output/configs/fusion_config.yaml")
print(f"  模型配置: output/configs/model_config.yaml")

print("\n" + "=" * 80)
print("✅ 配置创建完成！(研报 Baseline 版本)")
print("=" * 80)
print("\n🔴 研报 Baseline 关键特性:")
print(f"  1. 多因子输出: output_dim = {N_FACTORS} (N×F 矩阵)")
print(f"  2. 静态邻接矩阵: 基于行业分类构建")
print(f"  3. 后续集成: 对 F 个因子取平均或使用 LightGBM 加权")
print("\n  残差连接 (use_residual=True)")
print("     - TemporalBlock: LSTM输出 + 输入投影 → LayerNorm")
print("     - GraphBlock: GAT输出 + 输入投影 → LayerNorm")
print("     - 帮助梯度传播，防止深层网络梯度消失")
print("\n  学习率自动调整 (use_scheduler=True)")
print("     - 类型: ReduceLROnPlateau")
print("     - 当验证损失连续3个epoch无改善时，学习率降为原来的0.5倍")
print("     - 最小学习率: 1e-6")
print("=" * 80)

✅ DataManager v1.1.0 已加载

📝 步骤 1: 创建配置对象 (研报 Baseline 版本)

📌 研报 Baseline 配置:
   多因子输出维度 F = 64
   使用静态邻接矩阵 = False

1️⃣ 数据配置 (DataConfig)
✅ 数据配置创建成功
  数据文件: wind_processed_all.parquet
  窗口大小: 150
  批次大小: 1000 (全市场模式)
  划分策略: rolling
  划分比例: 0.7/0.2/0.1
组合模型配置摘要

输入特征维度: 7

【时序模块】
  - RNN类型: lstm
  - 隐藏层: 32
  - 层数: 2
  - 双向: False
  - 注意力: True
    类型: self
  - 输出维度: 32

【图模块】
  - GAT类型: standard
  - 隐藏维度: 32
  - 注意力头数: 4
  - 拼接多头: True
  - 输出维度: 32

【融合模块】
  - 融合策略: concat
  - 输入维度: 64
  - 隐藏层: [128, 64]
  - 激活函数: relu
  - BatchNorm: False
  - 残差连接: False
  - 输出维度: 64

【训练配置】
  - 设备: cuda
  - Epochs: 20
  - Batch Size: 1000
  - 学习率: 0.001
  - 优化器: adam
  - 损失函数: mse
  - 早停: 6

CompositeModelConfig(device=cuda, n_epochs=20, batch_size=1000, learning_rate=0.001, early_stop=6, optimizer=adam, loss_fn=mse, weight_decay=0.0, model_save_path=output/best_model.pth, log_dir=logs, verbose=True, seed=None, temporal=TemporalModuleConfig(enabled=True, name=None, rnn_type=lstm, hidden_size=32, num

/home/u2025210237/jupyterlab/quantclassic/model/modular_config.py:453: UserWarning: graph 模块已启用但未提供 adj_matrix_path
  warnings.warn("graph 模块已启用但未提供 adj_matrix_path")


In [2]:
# =============================================================================
# 步骤 2: 数据加载与准备
# =============================================================================

# 重新加载模块以确保使用最新代码
import sys
import importlib
from quantclassic.data_set import DataManager
import torch

# 1. 创建DataManager
print("\n1️⃣ 创建DataManager")
dm = DataManager(config=data_config)

# 2. 运行完整数据流程
print("\n2️⃣ 运行数据处理流程")
loaders = dm.run_full_pipeline(
    file_path= None,
    validate= False,
    auto_filter_features= False
    )

print(f"\n✅ 数据准备完成")
print(f"  特征维度: {len(dm.feature_cols)}")
print(f"  训练样本: {len(dm.datasets.train):,}")
print(f"  验证样本: {len(dm.datasets.val):,}")
print(f"  测试样本: {len(dm.datasets.test):,}")
print(f"  训练批次: {len(loaders.train)}")
print(f"  验证批次: {len(loaders.val)}")
print(f"  测试批次: {len(loaders.test)}")

# 3. 更新模型配置中的特征维度
input_dim = len(dm.feature_cols)
model_config.d_feat = input_dim

print(f"\n3️⃣ 更新模型配置")
print(f"  d_feat设置为: {input_dim}")

# 4. 显示前10个特征
print(f"\n4️⃣ 特征列表（前10个）:")
for i, col in enumerate(dm.feature_cols[:10], 1):
    print(f"  {i:2d}. {col}")
if len(dm.feature_cols) > 10:
    print(f"  ... 还有 {len(dm.feature_cols) - 10} 个特征")

# 5. ⚠️ 验证标签列是否被错误地用作特征
print(f"\n5️⃣ 数据泄露检查:")
label_col = data_config.label_col
if label_col in dm.feature_cols:
    print(f"  ❌ 错误：标签列 '{label_col}' 被用作特征（数据泄露）！")
else:
    print(f"  ✅ 正确：标签列 '{label_col}' 已正确排除")



2026-01-30 16:09:46 - DataManager - INFO - ✅ DataManager 初始化完成
INFO:DataManager:✅ DataManager 初始化完成
2026-01-30 16:09:46 - DataManager - INFO - 
INFO:DataManager:
2026-01-30 16:09:46 - DataManager - INFO - 🚀 开始完整数据处理流水线
INFO:DataManager:🚀 开始完整数据处理流水线
2026-01-30 16:09:46 - DataManager - INFO - ================================================================================

INFO:DataManager:================================================================================

2026-01-30 16:09:46 - DataManager - INFO - ================================================================================
INFO:DataManager:================================================================================
2026-01-30 16:09:46 - DataManager - INFO - 步骤 1/5: 加载原始数据
INFO:DataManager:步骤 1/5: 加载原始数据
2026-01-30 16:09:46 - DataManager - INFO - ================================================================================
INFO:DataManager:=========================================================================


1️⃣ 创建DataManager

2️⃣ 运行数据处理流程


2026-01-30 16:09:47,439 - DataLoader - INFO - 🔧 优化数据类型...
INFO:DataLoader:🔧 优化数据类型...
2026-01-30 16:09:54,230 - DataLoader - INFO -    内存优化: 3121.47MB → 763.52MB (减少 75.5%)
INFO:DataLoader:   内存优化: 3121.47MB → 763.52MB (减少 75.5%)
2026-01-30 16:09:55,141 - DataLoader - INFO - ✅ 数据加载完成: 14,547,107 行, 14 列
INFO:DataLoader:✅ 数据加载完成: 14,547,107 行, 14 列
2026-01-30 16:09:55 - DataManager - INFO - ================================================================================
INFO:DataManager:================================================================================
2026-01-30 16:09:55 - DataManager - INFO - 步骤 3/5: 特征工程
INFO:DataManager:步骤 3/5: 特征工程
2026-01-30 16:09:55 - DataManager - INFO - ================================================================================
INFO:DataManager:================================================================================
2026-01-30 16:09:55,479 - FeatureEngineer - INFO - ✅ 使用配置的特征列: 7 列
INFO:FeatureEngineer:✅ 使用配置的特征列: 7 列
2026-01-30 16:09


📊 数据摘要
形状: 14,547,107 行 × 14 列
内存占用: 763.52 MB
股票数量: 5,632
时间范围: 2000-05-25 00:00:00 ~ 2025-12-11 00:00:00

数据类型分布:
  float32: 11 列
  datetime64[ns]: 1 列
  category: 1 列
  category: 1 列

缺失值 (前10列):
  alpha_label: 457 (0.00%)


2026-01-30 16:10:05,303 - FeatureEngineer - INFO - 💾 特征信息已保存到: output
INFO:FeatureEngineer:💾 特征信息已保存到: output
2026-01-30 16:10:05 - DataManager - INFO - ================================================================================
INFO:DataManager:================================================================================
2026-01-30 16:10:05 - DataManager - INFO - 步骤 4/5: 数据划分
INFO:DataManager:步骤 4/5: 数据划分
2026-01-30 16:10:05 - DataManager - INFO - ================================================================================
INFO:DataManager:================================================================================
2026-01-30 16:10:06 - DataManager - INFO -    过滤标签缺失数据: 14,547,107 -> 14,546,650 (-457)
INFO:DataManager:   过滤标签缺失数据: 14,547,107 -> 14,546,650 (-457)
2026-01-30 16:10:06,874 - RollingWindowSplitter - INFO - 🔄 滚动窗口划分...
INFO:RollingWindowSplitter:🔄 滚动窗口划分...
2026-01-30 16:10:16,483 - RollingWindowSplitter - INFO -    生成 15 个滚动窗口
INFO:RollingWindowSplitter:   


📊 数据处理摘要
原始数据: 14,547,107 行
特征数量: 7

数据集:
  训练集: 40,492,269 样本
  验证集: 11,038,595 样本
  测试集: 3,239,677 样本

配置:
  窗口大小: 150
  批量大小: 1000
  划分策略: rolling


✅ 数据准备完成
  特征维度: 7
  训练样本: 40,492,269
  验证样本: 11,038,595
  测试样本: 3,239,677
  训练批次: 40493
  验证批次: 11039
  测试批次: 3240

3️⃣ 更新模型配置
  d_feat设置为: 7

4️⃣ 特征列表（前10个）:
   1. open
   2. high
   3. low
   4. close
   5. vol
   6. amount
   7. vwap

5️⃣ 数据泄露检查:
  ✅ 正确：标签列 'alpha_label' 已正确排除


In [3]:
# =============================================================================
# 步骤 2-DYNAMIC: 动态批次数据加载（按日构建batch + 动态邻接矩阵）
# =============================================================================
# 🔴 关键改动：
#   1. 使用 DailyBatchDataset 替代固定 batch_size
#   2. 每个batch是一个交易日的所有股票（动态大小）
#   3. 使用 HybridGraphBuilder 每日动态构建邻接矩阵
#
# 📌 与旧方法的区别：
#   旧方法：固定batch_size=1000，静态邻接矩阵
#   新方法：每日股票数决定batch_size，动态构建图
# =============================================================================

import sys
sys.path.insert(0, '/home/u2025210237/jupyterlab')
import pandas as pd
import torch
from pathlib import Path
from copy import deepcopy

# 导入动态图相关模块
from quantclassic.data_set.graph.daily_graph_loader import (
    DailyBatchDataset, DailyGraphDataLoader, create_daily_loader
)
from quantclassic.data_processor.graph_builder import (
    CorrGraphBuilder, IndustryGraphBuilder, HybridGraphBuilder,
    GraphBuilderFactory
)
# 🔴 修复：DynamicGraphTrainer 已废弃，改用 SimpleTrainer
from quantclassic.model.train import SimpleTrainer, TrainerConfig

print("\n" + "=" * 80)
print("📦 步骤 2-DYNAMIC: 动态批次数据加载")
print("=" * 80)

# =============================================================================
# 0. 🔴 重要：创建独立的动态配置副本（避免与静态流程冲突）
# =============================================================================
# 保存原始 d_feat（来自 DataManager 的处理后特征数量）
ORIGINAL_D_FEAT = model_config.d_feat if hasattr(model_config, 'd_feat') else len(dm.feature_cols)
print(f"\n⚠️ 保存原始 d_feat: {ORIGINAL_D_FEAT} (用于静态流程)")

# 创建动态流程专用配置副本
dynamic_model_config = deepcopy(model_config)

# =============================================================================
# 1. 加载原始数据
# =============================================================================
print("\n1️⃣ 加载数据...")
df_full = pd.read_parquet('output/wind_processed_all.parquet')
print(f"   数据形状: {df_full.shape}")
print(f"   时间范围: {df_full['trade_date'].min()} ~ {df_full['trade_date'].max()}")
print(f"   股票数量: {df_full['order_book_id'].nunique()}")

# =============================================================================
# 2. 配置参数
# =============================================================================
print("\n2️⃣ 配置参数...")

# 特征列（原始 7 个特征，DailyBatchDataset 内部会做窗口变换）
dynamic_feature_cols = ['open', 'high', 'low', 'close', 'vol', 'amount', 'vwap']
label_col = 'alpha_label'
stock_col = 'order_book_id'
time_col = 'trade_date'
window_size = 150  # 与原配置保持一致

# 🔴 更新动态配置的 d_feat（原始特征数量）
dynamic_model_config.d_feat = len(dynamic_feature_cols)

print(f"   特征列: {dynamic_feature_cols}")
print(f"   动态 d_feat: {dynamic_model_config.d_feat}")
print(f"   标签列: {label_col}")
print(f"   窗口大小: {window_size}")

# =============================================================================
# 2.5 🆕 构建全局股票-行业映射（解决动态图行业列缺失问题）
# =============================================================================
print("\n2.5️⃣ 构建股票-行业映射...")
if 'industry_code' in df_full.columns:
    stock_industry_mapping = dict(zip(df_full[stock_col], df_full['industry_code']))
    print(f"   ✅ 成功构建映射，共 {len(stock_industry_mapping)} 只股票")
else:
    stock_industry_mapping = {}
    print(f"   ⚠️ 未找到 industry_code 列，行业图将退化为自环")

# =============================================================================
# 3. 创建图构建器（混合图：行业 + 相关性）
# =============================================================================
print("\n3️⃣ 创建图构建器...")

# 方式1: 纯行业图（静态，推荐用于训练稳定性）
industry_builder = IndustryGraphBuilder(
    industry_col='industry_code',
    industry_adj_path='output/industry_adj_matrix.pt',
    stock_col=stock_col,
    stock_industry_mapping=stock_industry_mapping,  # 🆕 传入全局映射
    add_self_loop=True
)

# 方式2: 纯相关性图（动态，基于当日特征）
corr_builder = CorrGraphBuilder(
    method='cosine',
    top_k=10,
    stock_col=stock_col,
    add_self_loop=True
)

# 方式3: 混合图 = α * corr + (1-α) * industry
hybrid_builder = HybridGraphBuilder(
    alpha=0.7,
    corr_method='cosine',
    top_k=10,
    industry_col='industry_code',
    industry_adj_path='output/industry_adj_matrix.pt',
    stock_col=stock_col,
    stock_industry_mapping=stock_industry_mapping,  # 🆕 直接传入全局映射
    add_self_loop=True
)
print(f"   ✅ HybridGraphBuilder 已配置行业映射 ({len(stock_industry_mapping)} 只股票)")

# 选择使用的图构建器
USE_DYNAMIC_GRAPH = True
GRAPH_TYPE = 'hybrid'  # 'industry', 'corr', 'hybrid'

if USE_DYNAMIC_GRAPH:
    if GRAPH_TYPE == 'industry':
        graph_builder = industry_builder
        print(f"   图类型: 行业图 (静态)")
    elif GRAPH_TYPE == 'corr':
        graph_builder = corr_builder
        print(f"   图类型: 相关性图 (动态, top_k=10)")
    else:
        graph_builder = hybrid_builder
        print(f"   图类型: 混合图 (α=0.7, 动态)")
else:
    graph_builder = None
    print(f"   图类型: 无 (仅时序模型)")

# =============================================================================
# 4. 创建滚动窗口划分
# =============================================================================
print("\n4️⃣ 创建数据划分...")

# 按时间排序
df_full = df_full.sort_values([time_col, stock_col]).reset_index(drop=True)
df_full[time_col] = pd.to_datetime(df_full[time_col])

# 获取所有唯一日期
all_dates = sorted(df_full[time_col].unique())
n_dates = len(all_dates)

# 滚动窗口参数（与原配置对齐）
rolling_window_size = 2268  # 9年 = 252 * 9
rolling_step = 252  # 1年

# 计算窗口数量
n_windows = max(1, (n_dates - rolling_window_size) // rolling_step + 1)
print(f"   总交易日: {n_dates}")
print(f"   窗口大小: {rolling_window_size} 天")
print(f"   滚动步长: {rolling_step} 天")
print(f"   窗口数量: {n_windows}")

# =============================================================================
# 5. 为第一个窗口创建动态数据加载器（示例）
# =============================================================================
print("\n5️⃣ 创建动态数据加载器（窗口1示例）...")

# 第一个窗口的数据范围
window_idx = 0
start_idx = window_idx * rolling_step
end_idx = start_idx + rolling_window_size
train_end_idx = int(start_idx + rolling_window_size * 0.7)  # 70% 训练
val_end_idx = int(start_idx + rolling_window_size * 0.9)    # 20% 验证

# 提取窗口内的日期
train_dates = all_dates[start_idx:train_end_idx]
val_dates = all_dates[train_end_idx:val_end_idx]
test_dates = all_dates[val_end_idx:end_idx]

print(f"   训练日期: {len(train_dates)} 天 ({train_dates[0]} ~ {train_dates[-1]})")
print(f"   验证日期: {len(val_dates)} 天 ({val_dates[0]} ~ {val_dates[-1]})")
print(f"   测试日期: {len(test_dates)} 天 ({test_dates[0]} ~ {test_dates[-1]})")

# 划分数据
df_train = df_full[df_full[time_col].isin(train_dates)]
df_val = df_full[df_full[time_col].isin(val_dates)]
df_test = df_full[df_full[time_col].isin(test_dates)]

print(f"   训练样本: {len(df_train):,}")
print(f"   验证样本: {len(df_val):,}")
print(f"   测试样本: {len(df_test):,}")

# =============================================================================
# 6. 创建 DailyBatchDataset 和 DailyGraphDataLoader
# =============================================================================
print("\n6️⃣ 创建日批次数据集...")

# 计算有效标签的起始日期（需要 window_size 天历史）
valid_label_start_date = all_dates[window_size] if len(all_dates) > window_size else None

# 训练集
train_dataset = DailyBatchDataset(
    df=df_train,
    feature_cols=dynamic_feature_cols,
    label_col=label_col,
    window_size=window_size,
    time_col=time_col,
    stock_col=stock_col,
    enable_window_transform=True,
    window_price_log=True,
    window_volume_norm=True,
    price_cols=['open', 'high', 'low', 'close', 'vwap'],
    close_col='close',
    volume_cols=['vol', 'amount'],
    label_rank_normalize=True,
    label_rank_output_range=(-1, 1),
    valid_label_start_date=valid_label_start_date
)

# 验证集
val_dataset = DailyBatchDataset(
    df=df_val,
    feature_cols=dynamic_feature_cols,
    label_col=label_col,
    window_size=window_size,
    time_col=time_col,
    stock_col=stock_col,
    enable_window_transform=True,
    window_price_log=True,
    window_volume_norm=True,
    price_cols=['open', 'high', 'low', 'close', 'vwap'],
    close_col='close',
    volume_cols=['vol', 'amount'],
    label_rank_normalize=True,
    label_rank_output_range=(-1, 1)
)

# 测试集
test_dataset = DailyBatchDataset(
    df=df_test,
    feature_cols=dynamic_feature_cols,
    label_col=label_col,
    window_size=window_size,
    time_col=time_col,
    stock_col=stock_col,
    enable_window_transform=True,
    window_price_log=True,
    window_volume_norm=True,
    price_cols=['open', 'high', 'low', 'close', 'vwap'],
    close_col='close',
    volume_cols=['vol', 'amount'],
    label_rank_normalize=True,
    label_rank_output_range=(-1, 1)
)

print(f"   训练集有效天数: {len(train_dataset)}")
print(f"   验证集有效天数: {len(val_dataset)}")
print(f"   测试集有效天数: {len(test_dataset)}")

# =============================================================================
# 7. 创建 DailyGraphDataLoader
# =============================================================================
print("\n7️⃣ 创建日批次加载器（动态图）...")

device = 'cuda' if torch.cuda.is_available() else 'cpu'

use_pin_memory = False

# 训练加载器（打乱日期顺序）
train_daily_loader = DailyGraphDataLoader(
    dataset=train_dataset,
    graph_builder=graph_builder,
    feature_cols=dynamic_feature_cols,
    shuffle_dates=True,
    num_workers=0,
    pin_memory=use_pin_memory,
    device=device
)

# 验证加载器（保持日期顺序）
val_daily_loader = DailyGraphDataLoader(
    dataset=val_dataset,
    graph_builder=graph_builder,
    feature_cols=dynamic_feature_cols,
    shuffle_dates=False,
    num_workers=0,
    pin_memory=use_pin_memory,
    device=device
)

# 测试加载器（保持日期顺序）
test_daily_loader = DailyGraphDataLoader(
    dataset=test_dataset,
    graph_builder=graph_builder,
    feature_cols=dynamic_feature_cols,
    shuffle_dates=False,
    num_workers=0,
    pin_memory=use_pin_memory,
    device=device
)

print(f"   训练加载器: {len(train_daily_loader)} 个批次（天）")
print(f"   验证加载器: {len(val_daily_loader)} 个批次（天）")
print(f"   测试加载器: {len(test_daily_loader)} 个批次（天）")

# =============================================================================
# 8. 验证数据加载器输出
# =============================================================================
print("\n8️⃣ 验证数据加载器输出...")

# 获取一个batch并检查
for X, y, adj, stock_ids, date in train_daily_loader:
    print(f"   日期: {date}")
    print(f"   股票数量 (动态batch_size): {len(stock_ids)}")
    print(f"   X 形状: {X.shape}  (N, T, F)")
    print(f"   y 形状: {y.shape}  (N,)")
    if adj is not None:
        print(f"   adj 形状: {adj.shape}  (N, N)")
        print(f"   adj 非零边数: {(adj > 0).sum().item()}")
        # 🆕 验证行业边是否存在（非自环边）
        non_diag_edges = (adj > 0).sum().item() - adj.size(0)
        print(f"   adj 非自环边数: {non_diag_edges}")
    else:
        print(f"   adj: None (未使用图)")
    break

print("\n" + "=" * 80)
print("✅ 动态批次数据加载器创建完成!")
print("=" * 80)

print("\n🔴 动态批次模式关键特性:")
print(f"   1. 每日股票数决定batch_size (而非固定1000)")
print(f"   2. 邻接矩阵按日动态构建 (图类型: {GRAPH_TYPE})")
print(f"   3. 支持截面排名标准化标签")
print(f"   4. 窗口级价格对数变换 + 成交量标准化")

# 保存加载器到全局变量供后续使用
daily_loaders = {
    'train': train_daily_loader,
    'val': val_daily_loader,
    'test': test_daily_loader,
    'train_dataset': train_dataset,
    'val_dataset': val_dataset,
    'test_dataset': test_dataset,
    'graph_builder': graph_builder,
    'feature_cols': dynamic_feature_cols,
    'window_size': window_size,
    'model_config': dynamic_model_config,
    'original_d_feat': ORIGINAL_D_FEAT,
    'df_full': df_full,
    'all_dates': all_dates,
    'stock_industry_mapping': stock_industry_mapping,
    'rolling_config': {
        'rolling_window_size': rolling_window_size,
        'rolling_step': rolling_step,
        'n_windows': n_windows
    }
}
print("\n💡 加载器已保存到 'daily_loaders' 变量")
print(f"   ⚠️ 动态配置 d_feat = {dynamic_model_config.d_feat}")
print(f"   ⚠️ 原始配置 d_feat = {ORIGINAL_D_FEAT} (用于静态流程)")
print(f"\n💡 训练示例（使用 SimpleTrainer）：")
print(f"   trainer = SimpleTrainer(model, TrainerConfig(...), device=device)")
print(f"   trainer.fit(train_daily_loader, val_daily_loader, n_epochs=10)")
print(f"   predictions = trainer.predict(test_daily_loader)")

/home/u2025210237/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



📦 步骤 2-DYNAMIC: 动态批次数据加载

⚠️ 保存原始 d_feat: 7 (用于静态流程)

1️⃣ 加载数据...
   数据形状: (14547107, 14)
   时间范围: 2000-05-25 00:00:00 ~ 2025-12-11 00:00:00
   股票数量: 5632

2️⃣ 配置参数...
   特征列: ['open', 'high', 'low', 'close', 'vol', 'amount', 'vwap']
   动态 d_feat: 7
   标签列: alpha_label
   窗口大小: 150

2.5️⃣ 构建股票-行业映射...
   ✅ 成功构建映射，共 5632 只股票

3️⃣ 创建图构建器...
   ✅ HybridGraphBuilder 已配置行业映射 (5632 只股票)
   图类型: 混合图 (α=0.7, 动态)

4️⃣ 创建数据划分...
   总交易日: 6199
   窗口大小: 2268 天
   滚动步长: 252 天
   窗口数量: 16

5️⃣ 创建动态数据加载器（窗口1示例）...
   训练日期: 1587 天 (2000-05-25 00:00:00 ~ 2006-12-19 00:00:00)
   验证日期: 454 天 (2006-12-20 00:00:00 ~ 2008-11-03 00:00:00)
   测试日期: 227 天 (2008-11-04 00:00:00 ~ 2009-10-09 00:00:00)
   训练样本: 1,048,183
   验证样本: 561,110
   测试样本: 311,597

6️⃣ 创建日批次数据集...
   训练集有效天数: 1433
   验证集有效天数: 303
   测试集有效天数: 77

7️⃣ 创建日批次加载器（动态图）...
   训练加载器: 1433 个批次（天）
   验证加载器: 303 个批次（天）
   测试加载器: 77 个批次（天）

8️⃣ 验证数据加载器输出...
   日期: 2001-10-08 00:00:00
   股票数量 (动态batch_size): 2
   X 形状: torch.Size([2, 150, 7])  (N, T, F

In [ ]:
# =============================================================================
# 步骤 3-DYNAMIC: 使用 SimpleTrainer 训练动态图模型
# =============================================================================
# ✅ 已修复（2026-01-19）：
#   1. 使用 SimpleTrainer + TrainerConfig 替代已移除的 DynamicGraphTrainer
#   2. 支持日批次训练 + 动态邻接矩阵（adj 在 DataLoader 中构建）
#   3. 🆕 HybridGraphModel 自动检测 DailyGraphDataLoader 的 5 元素格式
#   4. 🆕 清除 stock_idx_position 避免将 adj 误判为 stock_idx
#   5. 使用独立的 dynamic_model_config，不影响静态流程
# =============================================================================

import sys
sys.path.insert(0, '/home/u2025210237/jupyterlab')

import pandas as pd
import numpy as np
import torch
from pathlib import Path

from quantclassic.model.train import SimpleTrainer, TrainerConfig
from quantclassic.model.hybrid_graph_models import HybridGraphModel

print("\n" + "=" * 80)
print("🚀 步骤 3-DYNAMIC: 动态图训练（SimpleTrainer）")
print("=" * 80)

# =============================================================================
# 1. 检查动态加载器是否已创建
# =============================================================================
if 'daily_loaders' not in dir():
    raise ValueError("❌ 请先运行上一个单元格创建 daily_loaders!")

train_daily_loader = daily_loaders['train']
val_daily_loader = daily_loaders['val']
test_daily_loader = daily_loaders['test']
dynamic_feature_cols = daily_loaders['feature_cols']

# 🔴 使用动态专用配置（不修改原始 model_config）
dynamic_model_config = daily_loaders['model_config']
ORIGINAL_D_FEAT = daily_loaders.get('original_d_feat', model_config.d_feat)

# 🆕 重要：清除 stock_idx_position，避免将 adj 误判为 stock_idx
# DailyGraphDataLoader 格式为 (X, y, adj, stock_ids, date)，batch[2] 是邻接矩阵
# 设置 stock_idx_position=2 会导致警告 "batch[2] 不是 long 类型张量"
dynamic_model_config.stock_idx_position = None
print(f"   🆕 已清除 stock_idx_position (动态图模式自动检测 batch adj)")

print(f"   训练批次: {len(train_daily_loader)} 天")
print(f"   验证批次: {len(val_daily_loader)} 天")
print(f"   测试批次: {len(test_daily_loader)} 天")

# =============================================================================
# 2. 确认模型配置（使用动态专用配置）
# =============================================================================
print("\n2️⃣ 模型配置...")
print(f"   动态 d_feat: {dynamic_model_config.d_feat}")
print(f"   output_dim (多因子数): {dynamic_model_config.fusion.output_dim}")
print(f"   ⚠️ 原始 model_config.d_feat 未被修改: {model_config.d_feat if 'model_config' in dir() else 'N/A'}")

# =============================================================================
# 3. 创建模型
# =============================================================================
print("\n3️⃣ 创建 HybridGraphModel...")

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 🔴 动态图模式：临时禁用静态邻接矩阵路径（邻接矩阵在 DataLoader 中动态构建）
original_adj_path = dynamic_model_config.adj_matrix_path
dynamic_model_config.adj_matrix_path = None  # 不预加载静态邻接矩阵

model_dynamic = HybridGraphModel.from_config(
    dynamic_model_config, 
    d_feat=dynamic_model_config.d_feat
)

# 恢复原始配置（以防后续其他代码需要）
dynamic_model_config.adj_matrix_path = original_adj_path

# 🔴 注意: HybridGraphModel 是 PyTorchModel 包装类，内部 .model 是 nn.Module
model_dynamic.model = model_dynamic.model.to(device)

print(f"   模型设备: {device}")
print(f"   模型参数量: {sum(p.numel() for p in model_dynamic.model.parameters()):,}")

# =============================================================================
# 4. 创建 SimpleTrainer（替代已移除的 DynamicGraphTrainer）
# =============================================================================
print("\n4️⃣ 创建 SimpleTrainer...")

trainer_config = TrainerConfig(
    n_epochs=dynamic_model_config.n_epochs,
    lr=dynamic_model_config.learning_rate,
    early_stop=dynamic_model_config.early_stop,
    optimizer=dynamic_model_config.optimizer,
    weight_decay=dynamic_model_config.weight_decay,
    
    # 学习率调度
    use_scheduler=dynamic_model_config.use_scheduler,
    scheduler_type=dynamic_model_config.scheduler_type,
    scheduler_patience=dynamic_model_config.scheduler_patience,
    scheduler_factor=dynamic_model_config.scheduler_factor,
    scheduler_min_lr=dynamic_model_config.scheduler_min_lr,
    
    # 损失函数
    loss_fn='mse',
    lambda_corr=dynamic_model_config.lambda_corr,  # 因子相关性正则化
    
    # 日志
    verbose=True,
    log_interval=10
)

dynamic_trainer = SimpleTrainer(
    model=model_dynamic.model,  # 传入内部的 nn.Module
    config=trainer_config,
    device=device
)

print(f"   训练轮数: {trainer_config.n_epochs}")
print(f"   学习率: {trainer_config.lr}")
print(f"   早停耐心: {trainer_config.early_stop}")
print(f"   相关性正则化: λ={trainer_config.lambda_corr}")

# =============================================================================
# 5. 开始训练
# =============================================================================
print("\n5️⃣ 开始动态图训练...")

save_path = 'output/saved_models/dynamic_graph_model.pth'
Path(save_path).parent.mkdir(parents=True, exist_ok=True)

training_results_dynamic = dynamic_trainer.train(
    train_loader=train_daily_loader,
    val_loader=val_daily_loader,
    n_epochs=trainer_config.n_epochs,
    save_path=save_path
)

# =============================================================================
# 6. 训练结果汇总
# =============================================================================
print("\n" + "=" * 80)
print("✅ 动态图训练完成!")
print("=" * 80)

print(f"\n📊 训练结果:")
print(f"   最佳 Epoch: {training_results_dynamic['best_epoch'] + 1}")
print(f"   最佳验证损失: {training_results_dynamic['best_score']:.6f}")
print(f"   训练耗时: {training_results_dynamic['elapsed_time']:.1f} 秒")

print(f"\n💾 模型已保存: {save_path}")

# =============================================================================
# 7. 测试集预测（🆕 使用 HybridGraphModel.predict - 支持动态 batch adj）
# =============================================================================
print("\n7️⃣ 测试集预测...")

# 🆕 HybridGraphModel.predict 现在自动检测 DailyGraphDataLoader 的 5 元素格式
# 并使用 batch 内的动态邻接矩阵进行图推理，不再出现 "batch[2] 不是 long 类型张量" 警告
model_dynamic.fitted = True  # 标记为已训练（因为是用 trainer 训练的）
predictions_raw = model_dynamic.predict(test_daily_loader, return_numpy=False)
print(f"   原始预测形状: {predictions_raw.shape}")

# 🆕 重建日期/股票索引（从 test_daily_loader 提取元数据）
pred_rows = []
label_rows = []
idx = 0

for batch in test_daily_loader:
    batch_x, batch_y, batch_adj, stock_ids, date = batch
    n_stocks = len(stock_ids)
    batch_preds = predictions_raw[idx:idx+n_stocks]
    idx += n_stocks
    
    date_i = pd.to_datetime(date)
    batch_y_cpu = batch_y.cpu().numpy()
    
    for i, stock in enumerate(stock_ids):
        pred_vec = batch_preds[i].numpy() if batch_preds[i].dim() >= 1 else batch_preds[i].unsqueeze(0).numpy()
        pred_vec = pred_vec.flatten()
        
        row = {
            'trade_date': date_i,
            'order_book_id': stock,
            'pred': float(pred_vec.mean())
        }
        # 添加多因子列
        for f_idx, val in enumerate(pred_vec):
            row[f'pred_factor_{f_idx}'] = float(val)
        pred_rows.append(row)
        
        label_rows.append({
            'trade_date': date_i,
            'order_book_id': stock,
            'label': float(batch_y_cpu[i])
        })

pred_df = pd.DataFrame(pred_rows)
label_df = pd.DataFrame(label_rows)

print(f"   预测结果形状: {pred_df.shape}")
print(f"   标签结果形状: {label_df.shape}")

# 合并预测和标签
test_predictions = pred_df.merge(
    label_df,
    on=['trade_date', 'order_book_id'],
    how='inner'
)
print(f"   合并后形状: {test_predictions.shape}")

# 计算测试集IC
test_ic = test_predictions.groupby('trade_date').apply(
    lambda x: x['pred'].corr(x['label']),
    include_groups=False
).mean()
print(f"\n📊 测试集 IC: {test_ic:.4f}")

# 保存到全局变量
dynamic_training_results = {
    'model': model_dynamic,
    'trainer': dynamic_trainer,
    'training_results': training_results_dynamic,
    'test_predictions': test_predictions,
    'test_ic': test_ic,
    'model_config': dynamic_model_config
}
print("\n💡 结果已保存到 'dynamic_training_results' 变量")

# 恢复原始 d_feat
model_config.d_feat = ORIGINAL_D_FEAT
print(f"   model_config.d_feat 已恢复为: {model_config.d_feat}")

2026-01-30 10:26:12 - HybridGraphModel - INFO - 初始化 PyTorchModel:
INFO:HybridGraphModel:初始化 PyTorchModel:
2026-01-30 10:26:12 - HybridGraphModel - INFO -   设备: cuda
INFO:HybridGraphModel:  设备: cuda
2026-01-30 10:26:12 - HybridGraphModel - INFO -   训练轮数: 20
INFO:HybridGraphModel:  训练轮数: 20
2026-01-30 10:26:12 - HybridGraphModel - INFO -   批量大小: 1000
INFO:HybridGraphModel:  批量大小: 1000
2026-01-30 10:26:12 - HybridGraphModel - INFO -   学习率: 0.001
INFO:HybridGraphModel:  学习率: 0.001
2026-01-30 10:26:12 - HybridGraphModel - INFO -   学习率调度器: plateau (patience=2, factor=0.5)
INFO:HybridGraphModel:  学习率调度器: plateau (patience=2, factor=0.5)
2026-01-30 10:26:12 - HybridGraphModel - INFO -   🆕 相关性正则化: lambda=0.01
INFO:HybridGraphModel:  🆕 相关性正则化: lambda=0.01



🚀 步骤 3-DYNAMIC: 动态图训练（SimpleTrainer）
   🆕 已清除 stock_idx_position (动态图模式自动检测 batch adj)
   训练批次: 1433 天
   验证批次: 303 天
   测试批次: 77 天

2️⃣ 模型配置...
   动态 d_feat: 7
   output_dim (多因子数): 64
   ⚠️ 原始 model_config.d_feat 未被修改: 7

3️⃣ 创建 HybridGraphModel...


2026-01-30 10:26:13 - HybridGraphModel - INFO - 🟢 动态图模式：未加载静态邻接矩阵，训练/推理将使用 batch 内提供的 adj。
INFO:HybridGraphModel:🟢 动态图模式：未加载静态邻接矩阵，训练/推理将使用 batch 内提供的 adj。
2026-01-30 10:26:13 - HybridGraphModel - INFO - HybridGraph 模型参数:
INFO:HybridGraphModel:HybridGraph 模型参数:
2026-01-30 10:26:13 - HybridGraphModel - INFO -   特征维度: 7
INFO:HybridGraphModel:  特征维度: 7
2026-01-30 10:26:13 - HybridGraphModel - INFO -   RNN类型: lstm, 隐藏层: 32, 层数: 2
INFO:HybridGraphModel:  RNN类型: lstm, 隐藏层: 32, 层数: 2
2026-01-30 10:26:13 - HybridGraphModel - INFO -   使用Attention: True
INFO:HybridGraphModel:  使用Attention: True
2026-01-30 10:26:13 - HybridGraphModel - INFO -   使用图网络: True
INFO:HybridGraphModel:  使用图网络: True
2026-01-30 10:26:13 - HybridGraphModel - INFO -   🆕 残差连接: True
INFO:HybridGraphModel:  🆕 残差连接: True
2026-01-30 10:26:13 - HybridGraphModel - INFO -   GAT类型: standard, 隐藏层: 32, 头数: 4
INFO:HybridGraphModel:  GAT类型: standard, 隐藏层: 32, 头数: 4
2026-01-30 10:26:13 - HybridGraphModel - INFO -   MLP隐藏层: [128, 64]
INFO

   模型设备: cuda
   模型参数量: 36,577

4️⃣ 创建 SimpleTrainer...
   训练轮数: 20
   学习率: 0.001
   早停耐心: 6
   相关性正则化: λ=0.01

5️⃣ 开始动态图训练...


2026-01-30 10:31:42 - SimpleTrainer - INFO - Epoch 1/20 - Train Loss: 0.577480, Val Loss: 0.333983
INFO:SimpleTrainer:Epoch 1/20 - Train Loss: 0.577480, Val Loss: 0.333983
2026-01-30 10:31:42 - SimpleTrainer - INFO - 💾 检查点已保存: output/saved_models/dynamic_graph_model.pth
INFO:SimpleTrainer:💾 检查点已保存: output/saved_models/dynamic_graph_model.pth
2026-01-30 10:37:13 - SimpleTrainer - INFO - Epoch 2/20 - Train Loss: 0.577698, Val Loss: 0.334010
INFO:SimpleTrainer:Epoch 2/20 - Train Loss: 0.577698, Val Loss: 0.334010
2026-01-30 10:42:42 - SimpleTrainer - INFO - Epoch 3/20 - Train Loss: 0.577654, Val Loss: 0.333974
INFO:SimpleTrainer:Epoch 3/20 - Train Loss: 0.577654, Val Loss: 0.333974
2026-01-30 10:42:42 - SimpleTrainer - INFO - 💾 检查点已保存: output/saved_models/dynamic_graph_model.pth
INFO:SimpleTrainer:💾 检查点已保存: output/saved_models/dynamic_graph_model.pth
2026-01-30 10:48:12 - SimpleTrainer - INFO - Epoch 4/20 - Train Loss: 0.577643, Val Loss: 0.333985
INFO:SimpleTrainer:Epoch 4/20 - Train Los

Epoch 00004: reducing learning rate of group 0 to 5.0000e-04.


2026-01-30 10:53:41 - SimpleTrainer - INFO - Epoch 5/20 - Train Loss: 0.577478, Val Loss: 0.333975
INFO:SimpleTrainer:Epoch 5/20 - Train Loss: 0.577478, Val Loss: 0.333975


In [ ]:
# =============================================================================
# 步骤 3-DYNAMIC: 滚动窗口 + 动态图训练（DataManager + RollingDailyTrainer 重构版）
# =============================================================================
# 🔴 重构说明：
#   - 数据加载器构建已下沉到 DataManager.create_rolling_daily_loaders_from_test()
#   - 消除 Notebook 中的胶水代码（~60行手写函数 → 1行调用）
#   - 训练使用 RollingDailyTrainer（自动权重继承、显存回收）
# =============================================================================

import torch, gc, sys
sys.path.insert(0, '/home/u2025210237/jupyterlab')

import importlib
import quantclassic.model.train.rolling_daily_trainer
importlib.reload(quantclassic.model.train.rolling_daily_trainer)
import quantclassic.data_set.manager
importlib.reload(quantclassic.data_set.manager)

from quantclassic.model.train.rolling_daily_trainer import RollingDailyTrainer, DailyRollingConfig
from quantclassic.model import HybridGraphModel
from pathlib import Path
import numpy as np, pandas as pd

print("=" * 80)
print("🚀 步骤 3-DYNAMIC: 滚动窗口 + 动态图训练（DataManager + RollingDailyTrainer 重构版）")
print("=" * 80)

# =============================================================================
# 1. 配置参数（统一取自 data_config / model_config）
# =============================================================================

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"📌 设备: {device}")

# 动态图专用配置（深拷贝，避免污染原配置）
ORIGINAL_D_FEAT = model_config.d_feat
dynamic_feature_cols = dm.feature_cols.copy()
dynamic_model_config = model_config.copy()  # BaseConfig.copy() 深拷贝
dynamic_model_config.d_feat = len(dynamic_feature_cols)

print(f"\n📌 动态图配置:")
print(f"   动态 d_feat: {dynamic_model_config.d_feat}")
print(f"   原始 d_feat: {ORIGINAL_D_FEAT} (静态流程使用)")

# 滚动参数统一取自 data_config
window_size = data_config.window_size
rolling_window_size = data_config.rolling_window_size
rolling_step = data_config.rolling_step
val_ratio = data_config.val_ratio / (data_config.train_ratio + data_config.val_ratio)

print(f"\n📌 参数来源: data_config (正式配置)")
print(f"   window_size={window_size}, rolling_window_size={rolling_window_size}, rolling_step={rolling_step}")
print(f"   val_ratio={val_ratio:.2f}")

# =============================================================================
# 2. 使用 DataManager 创建滚动窗口数据加载器（🔴 核心改动：1行替代60行）
# =============================================================================

print("\n2️⃣ 使用 DataManager.create_rolling_daily_loaders_from_test() 创建数据加载器...")

# Switch to create_rolling_daily_loaders to use all rolling windows
gb_config = getattr(graph_builder, 'config', None)
if hasattr(gb_config, 'to_dict'): gb_config = gb_config.to_dict()

window_loaders = dm.create_rolling_daily_loaders(
    graph_builder_config=gb_config,
    val_ratio=val_ratio,
    device=device,
)

print(f"   创建了 {len(window_loaders)} 个窗口数据加载器")

# =============================================================================
# 3. 使用 RollingDailyTrainer 进行滚动窗口训练
# =============================================================================

print("\n3️⃣ 使用 RollingDailyTrainer 进行滚动窗口训练...")

# 定义 model_factory 闭包
def model_factory():
    """模型工厂：每次调用返回新的 HybridNet 模型（动态图模式）"""
    original_adj_path = dynamic_model_config.adj_matrix_path
    dynamic_model_config.adj_matrix_path = None
    model_wrapper = HybridGraphModel.from_config(dynamic_model_config, d_feat=dynamic_model_config.d_feat)
    dynamic_model_config.adj_matrix_path = original_adj_path
    return model_wrapper.model.to(device)

# 构造 DailyRollingConfig（复用 model_config 的训练参数）
N_EPOCHS = dynamic_model_config.n_epochs
EARLY_STOP = dynamic_model_config.early_stop
LEARNING_RATE = dynamic_model_config.learning_rate

rolling_config = DailyRollingConfig(
    n_epochs=N_EPOCHS,
    lr=LEARNING_RATE,
    early_stop=EARLY_STOP,
    weight_inheritance=True,           # ✅ 自动继承上一窗口权重
    save_each_window=True,             # ✅ 自动保存每个窗口模型
    gc_interval=1,                     # ✅ 每窗口进行显存回收
    offload_to_cpu=True,               # ✅ 切窗时模型移到 CPU
    clear_cache_on_window_end=True,    # ✅ 窗口结束清理 CUDA 缓存
    reset_optimizer=True,
    reset_scheduler=True,
    optimizer='adam',
    use_scheduler=True,
    scheduler_type='plateau',
    scheduler_patience=2,
    scheduler_factor=0.5,
    loss_fn='mse',
    verbose=False,
    log_interval=10,
)

print(f"\n📌 训练参数 (来自 model_config):")
print(f"   n_epochs={N_EPOCHS}, early_stop={EARLY_STOP}, lr={LEARNING_RATE}")
print(f"   weight_inheritance=True, gc_interval=1, offload_to_cpu=True")

# 创建训练器
rolling_trainer = RollingDailyTrainer(
    model_factory=model_factory,
    config=rolling_config,
    device=device,
)

# 执行训练
save_dir = Path('output/rolling_models_dynamic')
save_dir.mkdir(parents=True, exist_ok=True)

print(f"\n   保存目录: {save_dir}")
print(f"   开始训练 {len(window_loaders)} 个窗口...")

results = rolling_trainer.train(
    rolling_loaders=window_loaders,
    save_dir=str(save_dir),
)

# =============================================================================
# 4. 汇总结果（从 trainer 获取预测并计算 IC）
# =============================================================================

print("\n" + "=" * 80)
print("✅ 滚动窗口动态图训练完成!")
print("=" * 80)

# 从 trainer 获取合并的预测结果
all_predictions_df = rolling_trainer.get_all_predictions()

# 列名对齐（trainer 返回 y_true，回测系统需要 label）
if 'y_true' in all_predictions_df.columns:
    all_predictions_df = all_predictions_df.rename(columns={'y_true': 'label'})

# 计算每个窗口的 IC
test_ics = []
for w_result in results.get('window_results', []):
    if w_result.predictions is not None and len(w_result.predictions) > 0:
        pred_df = w_result.predictions
        if 'y_true' in pred_df.columns:
            pred_df = pred_df.rename(columns={'y_true': 'label'})
        if 'label' in pred_df.columns and 'trade_date' in pred_df.columns:
            window_ic = pred_df.groupby('trade_date').apply(
                lambda x: x['pred'].corr(x['label']), include_groups=False
            ).mean()
            if pd.notna(window_ic):
                test_ics.append(window_ic)
                print(f"   窗口 {w_result.window_id} 测试IC: {window_ic:.4f}")

print(f"\n📊 汇总统计 ({len(test_ics)} 个窗口):")
if test_ics:
    print(f"   平均测试IC: {np.mean(test_ics):.4f}")
    print(f"   IC标准差: {np.std(test_ics):.4f}")
    print(f"   最大IC: {np.max(test_ics):.4f}")
    print(f"   最小IC: {np.min(test_ics):.4f}")
else:
    print("   无有效 IC 数据")
print(f"   总预测样本: {len(all_predictions_df):,}")
print(f"   总训练耗时: {results.get('elapsed_time', 0):.1f}s ({results.get('elapsed_time', 0)/60:.1f} min)")

# =============================================================================
# 5. 转换为回测系统兼容格式
# =============================================================================

print("\n5️⃣ 转换为回测系统兼容格式...")

if len(all_predictions_df) > 0:
    all_predictions_df['trade_date'] = pd.to_datetime(all_predictions_df['trade_date'])
    
    # 合并真实收益（用于回测）
    df_full_for_returns = pd.concat([dm._train_df, dm._val_df, dm._test_df], ignore_index=True)
    return_col_name = 'y_ret_10d' if 'y_ret_10d' in df_full_for_returns.columns else dm.config.label_col
    df_returns = df_full_for_returns[['trade_date', 'order_book_id', return_col_name]].copy()
    df_returns['trade_date'] = pd.to_datetime(df_returns['trade_date'])
    df_returns = df_returns.rename(columns={return_col_name: 'future_return'})
    all_predictions_df = all_predictions_df.merge(df_returns, on=['trade_date', 'order_book_id'], how='left')
    
    # 列名对齐回测系统
    all_predictions_df = all_predictions_df.rename(columns={'pred': 'factor_raw_std'})
    
    print(f"   日期范围: {all_predictions_df['trade_date'].min()} ~ {all_predictions_df['trade_date'].max()}")
    print(f"   真实收益列: future_return (非空: {all_predictions_df['future_return'].notna().sum():,})")

# =============================================================================
# 6. 保存结果到全局变量
# =============================================================================

rolling_dynamic_results = {
    'window_results': results.get('window_results', []),
    'all_predictions': all_predictions_df,
    'test_ics': test_ics,
    'trainer': rolling_trainer,
    'config': {
        'window_size': window_size,
        'rolling_window_size': rolling_window_size,
        'rolling_step': rolling_step,
        'graph_type': 'hybrid',
        'use_dynamic_graph': True,
        'd_feat': dynamic_model_config.d_feat,
        'weight_inheritance': True,
    }
}

print("\n💡 结果已保存到 'rolling_dynamic_results' 变量")
print("   - rolling_dynamic_results['all_predictions']: 合并预测 DataFrame")
print("   - rolling_dynamic_results['trainer']: RollingDailyTrainer 实例")
print("   - rolling_dynamic_results['test_ics']: 各窗口 IC 列表")

# 恢复原始 d_feat
model_config.d_feat = ORIGINAL_D_FEAT
print(f"\n6️⃣ model_config.d_feat 已恢复为: {model_config.d_feat}")

print("\n" + "=" * 80)
print("🎯 下一步: 使用 rolling_dynamic_results['all_predictions'] 进行回测")
print("=" * 80)

🚀 步骤 3-DYNAMIC: 滚动窗口 + 动态图训练（DataManager + RollingDailyTrainer 重构版）
📌 设备: cuda

📌 动态图配置:
   动态 d_feat: 7
   原始 d_feat: 7 (静态流程使用)

📌 参数来源: data_config (正式配置)
   window_size=150, rolling_window_size=2268, rolling_step=252
   val_ratio=0.22

2️⃣ 使用 DataManager.create_rolling_daily_loaders_from_test() 创建数据加载器...


2026-01-30 16:40:25 - DataManager - INFO -   已构建全局股票-行业映射: 5632 只股票
INFO:DataManager:  已构建全局股票-行业映射: 5632 只股票
2026-01-30 16:40:25 - DataManager - INFO - 
INFO:DataManager:
2026-01-30 16:40:25 - DataManager - INFO - 🔄 创建滚动窗口日批次加载器
INFO:DataManager:🔄 创建滚动窗口日批次加载器
2026-01-30 16:40:25 - DataManager - INFO - ================================================================================
INFO:DataManager:================================================================================
2026-01-30 16:40:25 - DataManager - INFO -   总窗口数: 15
INFO:DataManager:  总窗口数: 15
/home/u2025210237/jupyterlab/quantclassic/data_set/graph/daily_graph_loader.py:204: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for stock, group in self.df.groupby(self.stock_col, observed=False):
/home/u2025210237/jupyterlab/quantclassic/d

In [ ]:
# =============================================================================
# 步骤 4-DYNAMIC: 动态图模型多因子回测
# =============================================================================
# 🔴 修复后的回测流程：
#   1. 使用 MultiFactorBacktest 处理 64 个因子列
#   2. 使用真实收益列 future_return（而非截面排名标签）
#   3. 支持 IC 加权集成
# =============================================================================

import sys
sys.path.insert(0, '/home/u2025210237/jupyterlab')

import importlib
import quantclassic.backtest
importlib.reload(quantclassic.backtest)

from quantclassic.backtest import BacktestConfig, MultiFactorBacktest, FactorBacktestSystem
from pathlib import Path

print("=" * 80)
print("🚀 步骤 4-DYNAMIC: 动态图模型多因子回测")
print("=" * 80)

# =============================================================================
# 1. 检查动态训练结果是否存在
# =============================================================================
if 'rolling_dynamic_results' not in dir():
    raise ValueError(
        "❌ 未找到 rolling_dynamic_results！\n"
        "   请先运行步骤 3.5-DYNAMIC（滚动窗口动态图训练）"
    )

# 获取动态训练的预测结果
dynamic_predictions = rolling_dynamic_results['all_predictions'].copy()
factor_cols = rolling_dynamic_results.get('factor_cols', [])

print(f"\n📊 动态预测结果:")
print(f"   数据形状: {dynamic_predictions.shape}")
print(f"   时间范围: {dynamic_predictions['trade_date'].min()} ~ {dynamic_predictions['trade_date'].max()}")
print(f"   股票数量: {dynamic_predictions['order_book_id'].nunique()}")
print(f"   多因子列数: {len(factor_cols)}")

# =============================================================================
# 2. 确定回测模式（多因子 vs 单因子）
# =============================================================================
USE_MULTIFACTOR = len(factor_cols) > 0
print(f"\n📌 回测模式: {'多因子 IC 加权' if USE_MULTIFACTOR else '单因子'}")

# =============================================================================
# 3. 回测参数配置
# =============================================================================
output_dir = 'output/backtest_dynamic_graph'
Path(output_dir).mkdir(parents=True, exist_ok=True)

backtest_options = dict(
    # ===== 输出目录 =====
    output_dir=output_dir,
    
    # ===== 因子处理 =====
    winsorize_method='quantile',
    winsorize_quantiles=(0.025, 0.975),
    standardize_method='zscore',
    fillna_method='median',
    
    # ===== 组合设置 =====
    n_groups=10,
    rebalance_freq='biweekly',
    weight_method='equal',
    long_ratio=0.2,
    short_ratio=0.2,
    
    # ===== 成本设置 =====
    consider_cost=True,
    commission_rate=0.0003,
    stamp_tax_rate=0.001,
    slippage_rate=0.001,
    
    # ===== 报告设置 =====
    save_plots=True,
    generate_excel=True,
    report_title='动态图模型回测报告（日批次+动态邻接矩阵）',
)

print(f"\n【回测参数】")
print(f"   输出目录: {output_dir}")
print(f"   分组数量: {backtest_options['n_groups']}")
print(f"   调仓频率: {backtest_options['rebalance_freq']}")

# =============================================================================
# 4. 确定收益列
# =============================================================================
# 🔴 优先使用 future_return（真实收益），否则回退到 label（截面排名）
if 'future_return' in dynamic_predictions.columns:
    return_col = 'future_return'
    print(f"\n✅ 使用真实收益列: {return_col}")
elif 'label' in dynamic_predictions.columns:
    return_col = 'label'
    print(f"\n⚠️ 未找到 future_return，使用截面排名标签: {return_col}")
else:
    raise ValueError("❌ 缺少收益列！需要 'future_return' 或 'label'")

# =============================================================================
# 5. 计算 IC 统计
# =============================================================================
print(f"\n4️⃣ 计算 IC 统计...")

# 单因子 IC（均值因子）
single_factor_col = 'factor_raw_std'
ic_by_date = dynamic_predictions.groupby('trade_date').apply(
    lambda x: x[single_factor_col].corr(x[return_col]) if return_col in x.columns else None
)
ic_by_date = ic_by_date.dropna()

print(f"\n📈 单因子（均值）IC 统计:")
print(f"   IC 均值: {ic_by_date.mean():.4f}")
print(f"   IC 标准差: {ic_by_date.std():.4f}")
print(f"   ICIR: {ic_by_date.mean() / ic_by_date.std() if ic_by_date.std() > 0 else 0:.4f}")
print(f"   IC 胜率: {(ic_by_date > 0).mean():.2%}")

# =============================================================================
# 6. 运行回测
# =============================================================================
backtest_config = BacktestConfig(**backtest_options)

if USE_MULTIFACTOR:
    # 🔴 多因子回测（IC 加权集成）
    print(f"\n5️⃣ 运行多因子回测 ({len(factor_cols)} 个因子)...")
    
    backtest = MultiFactorBacktest(backtest_config)
    
    backtest_results = backtest.run(
        dynamic_predictions,
        stock_col='order_book_id',
        time_col='trade_date',
        label_col=return_col,
        ensemble_method='ic_weighted',  # IC 加权集成
        save_results=True,
    )
    
    # 显示因子权重
    weights = backtest_results.get('ensemble_weights', {})
    if weights:
        print(f"\n⚖️ 因子权重 (Top 10):")
        for name, w in sorted(weights.items(), key=lambda x: -abs(x[1]))[:10]:
            print(f"   {name}: {w:.4f}")
else:
    # 单因子回测
    print(f"\n5️⃣ 运行单因子回测...")
    
    backtest_system = FactorBacktestSystem(backtest_config)
    
    backtest_results = backtest_system.run_backtest(
        data_df=dynamic_predictions,
        factor_col=single_factor_col,
        return_col=return_col,
        stock_col='order_book_id',
        time_col='trade_date',
        save_results=True
    )

# =============================================================================
# 7. 结果汇总
# =============================================================================
print(f"\n" + "=" * 80)
print(f"✅ 动态图模型回测完成!")
print("=" * 80)

# IC 统计
if 'ic_stats' in backtest_results:
    ic_stats = backtest_results['ic_stats']
    print(f"\n📊 IC 统计:")
    print(f"   IC 均值:  {ic_stats.get('ic_mean', 'N/A')}")
    print(f"   ICIR:     {ic_stats.get('icir', 'N/A')}")
    print(f"   IC 胜率:  {ic_stats.get('ic_win_rate', 'N/A')}")

# 绩效指标
if 'metrics' in backtest_results:
    metrics = backtest_results['metrics']
    print(f"\n📈 组合绩效:")
    if isinstance(metrics, dict):
        if 'long_short' in metrics:
            ls = metrics['long_short']
            print(f"\n   [多空组合]:")
            print(f"      年化收益: {ls.get('annual_return', 0):.2%}")
            print(f"      年化波动: {ls.get('annual_volatility', 0):.2%}")
            print(f"      夏普比率: {ls.get('sharpe_ratio', 0):.4f}")
            print(f"      最大回撤: {ls.get('max_drawdown', 0):.2%}")
        else:
            for key, value in metrics.items():
                if isinstance(value, dict):
                    print(f"\n   [{key}]:")
                    for k, v in value.items():
                        if isinstance(v, float):
                            print(f"      {k}: {v:.4f}")

# 窗口 IC 对比
print(f"\n📊 各窗口 IC 对比:")
for r in rolling_dynamic_results['window_results']:
    ic = r.get('test_ic', None)
    if ic is not None:
        print(f"   窗口 {r['window_idx']+1}: IC = {ic:.4f}")

# 提示
print(f"\n💡 提示:")
print(f"   • 回测结果已保存到: {output_dir}")
print(f"   • 回测模式: {'多因子 IC 加权' if USE_MULTIFACTOR else '单因子'}")
print(f"   • 收益列: {return_col}")
print(f"   • 可与静态模型回测结果（output/backtest_multifactor）对比")

# 保存到全局变量
dynamic_backtest_results = backtest_results
print(f"\n💾 结果已保存到 'dynamic_backtest_results' 变量")


In [ ]:
# =============================================================================
# 步骤 2.5: 构建静态行业邻接矩阵（研报 Baseline）
# =============================================================================
# 🔴 研报要求：
#   - 使用【静态行业邻接矩阵】而非动态相关性图
#   - 同行业股票之间 A[i,j] = 1，不同行业 A[i,j] = 0
#   - 这避免了使用目标列（alpha_label）计算相关性导致的数据泄露
#
# 📌 与旧方法的区别：
#   旧方法（已废弃）：每个窗口用 alpha_label 相关性动态构建邻接矩阵
#   新方法（研报）：一次性构建行业邻接矩阵，所有窗口共用
# =============================================================================

import pandas as pd
import torch
from pathlib import Path
from quantclassic.model.build_industry_adj import (
    build_industry_adjacency_matrix,
    build_correlation_adjacency_matrix,
    load_adjacency_matrix
)

print("\n" + "=" * 80)
print("🔗 步骤 2.5: 构建静态行业邻接矩阵（研报 Baseline）")
print("=" * 80)

# 加载处理后的数据（与训练数据保持一致）
df_adj = pd.read_parquet('output/wind_processed_all.parquet')

print(f"\n📊 数据概览:")
print(f"   样本数: {len(df_adj):,}")
print(f"   股票数: {df_adj['order_book_id'].nunique()}")
print(f"   行业数: {df_adj['industry_name'].nunique()}")

# 🔴 研报 Baseline: 基于行业分类构建静态邻接矩阵
adj_save_path = 'output/industry_adj_matrix.pt'

if USE_STATIC_ADJ:
    print(f"\n📌 构建行业邻接矩阵...")
    adj_matrix, stock_list, stock_to_idx = build_industry_adjacency_matrix(
        df=df_adj,
        stock_col='order_book_id',
        industry_col='industry_code',  # 🔴 修复：使用实际存在的列名
        save_path=adj_save_path,
        add_self_loop=True,
        normalize=False  # GAT 内部会做 softmax 归一化
    )
    
    print(f"\n✅ 行业邻接矩阵构建完成!")
    print(f"   矩阵形状: {adj_matrix.shape}")
    print(f"   股票数量: {len(stock_list)}")
    print(f"   保存路径: {adj_save_path}")
    
    # 更新模型配置中的邻接矩阵路径
    model_config.adj_matrix_path = adj_save_path
    
else:
    # 🔴 备选方案：基于历史收益率相关性（使用 close 而非 alpha_label）
    print(f"\n📌 构建相关性邻接矩阵（使用历史收益率）...")
    adj_matrix, stock_list, stock_to_idx = build_correlation_adjacency_matrix(
        df=df_adj,
        stock_col='order_book_id',
        time_col='trade_date',
        return_col='close',  # 🔴 使用收盘价收益率，不是目标列！
        top_k=10,
        method='pearson',
        save_path='output/correlation_adj_matrix.pt'
    )
    model_config.adj_matrix_path = 'output/correlation_adj_matrix.pt'

# 配置邻接矩阵参数（用于 RollingWindowTrainer）
# 🔴 研报 Baseline: 不再动态构建邻接矩阵
adj_config = {
    'use_static': True,  # 🔴 使用静态邻接矩阵
    'static_adj_path': adj_save_path,
    'stock_list': stock_list,
    'stock_to_idx': stock_to_idx
}

print("\n" + "=" * 80)
print("✅ 邻接矩阵配置完成！（研报 Baseline）")
print("=" * 80)

print("\n💡 研报 Baseline 说明:")
print("   • 使用静态行业邻接矩阵（所有窗口共用）")
print("   • 同行业股票互相连接，不同行业不连接")
print("   • 避免使用目标列计算相关性导致的数据泄露")
print(f"   • 矩阵保存路径: {adj_save_path}")
print("\n🔄 下一步: 运行下一个 Cell 开始训练")


🔗 步骤 2.5: 构建静态行业邻接矩阵（研报 Baseline）

📊 数据概览:
   样本数: 2,235,132
   股票数: 800
   行业数: 31

📌 构建行业邻接矩阵...

📊 数据概览:
   样本数: 2,235,132
   股票数: 800
   行业数: 31

📌 构建行业邻接矩阵...

✅ 行业邻接矩阵构建完成!
   矩阵形状: torch.Size([800, 800])
   股票数量: 800
   保存路径: output/industry_adj_matrix.pt

✅ 邻接矩阵配置完成！（研报 Baseline）

💡 研报 Baseline 说明:
   • 使用静态行业邻接矩阵（所有窗口共用）
   • 同行业股票互相连接，不同行业不连接
   • 避免使用目标列计算相关性导致的数据泄露
   • 矩阵保存路径: output/industry_adj_matrix.pt

🔄 下一步: 运行下一个 Cell 开始训练

✅ 行业邻接矩阵构建完成!
   矩阵形状: torch.Size([800, 800])
   股票数量: 800
   保存路径: output/industry_adj_matrix.pt

✅ 邻接矩阵配置完成！（研报 Baseline）

💡 研报 Baseline 说明:
   • 使用静态行业邻接矩阵（所有窗口共用）
   • 同行业股票互相连接，不同行业不连接
   • 避免使用目标列计算相关性导致的数据泄露
   • 矩阵保存路径: output/industry_adj_matrix.pt

🔄 下一步: 运行下一个 Cell 开始训练


In [6]:
# =============================================================================
# 步骤 3: 滚动窗口训练 (Walk-Forward Training) - 研报 Baseline
# =============================================================================
# 🔴 研报 Baseline 修改：
#   1. 使用静态行业邻接矩阵（不再动态构建）
#   2. 模型输出 N×F 因子矩阵
#
# 🆕 智能训练：
#   - 检测磁盘是否已有完整模型，有则跳过训练直接恢复
#   - 无需重复训练，节省时间
# =============================================================================

import sys
import importlib
from pathlib import Path

from quantclassic.data_manager.rolling_trainer import RollingWindowTrainer
from quantclassic.model.hybrid_graph_models import HybridGraphModel

print("\n" + "=" * 80)
print("🔄 步骤 3: 滚动窗口训练 (研报 Baseline)")
print("=" * 80)

print(f"\n📌 研报 Baseline 配置:")
print(f"   多因子输出维度 F = {N_FACTORS}")
print(f"   静态邻接矩阵: {model_config.adj_matrix_path}")

# =============================================================================
# 1. 创建滚动窗口训练器
# =============================================================================
trainer = dm.create_rolling_window_trainer()

# =============================================================================
# 2. 检测是否需要训练（智能恢复）
# =============================================================================
save_dir = Path('output/rolling_models')
VAL_RATIO = 2/9  # 7:2:1 划分，验证集占训练+验证的 2/9

# 检测已有模型数量
existing_models = list(save_dir.glob('window_*_model.pth')) if save_dir.exists() else []
have_all_models = len(existing_models) >= trainer.n_windows

print(f"\n📂 模型检测:")
print(f"   保存目录: {save_dir}")
print(f"   需要窗口: {trainer.n_windows}")
print(f"   已有模型: {len(existing_models)}")

if have_all_models:
    # =============================================================================
    # 2A. 从磁盘恢复（跳过训练）
    # =============================================================================
    print(f"\n✓ 检测到完整模型，跳过训练，直接恢复...")
    
    training_results = []
    for i in range(trainer.n_windows):
        model_path = save_dir / f'window_{i+1}_model.pth'
        
        # 重建测试数据集
        _, _, test_dataset = trainer.create_datasets_for_window(i, val_ratio=VAL_RATIO)
        
        # 加载模型
        model = HybridGraphModel.from_config(model_config, d_feat=len(trainer.feature_cols))
        model.load_model(str(model_path))
        
        training_results.append({
            'window_idx': i,
            'model': model,
            'test_dataset': test_dataset,
            'save_path': str(model_path),
            'train_loss': None,
            'val_loss': None,
            'best_epoch': model.best_epoch,
            'best_score': model.best_score
        })
        
        print(f"   窗口 {i+1}/{trainer.n_windows}: 已恢复")
    
    # 🆕 同步到 trainer.window_results（供后续 get_summary 等方法使用）
    trainer.window_results = training_results
    
    print(f"\n✅ 成功恢复 {len(training_results)} 个窗口模型（无需重新训练）")
    
else:
    # =============================================================================
    # 2B. 执行训练
    # =============================================================================
    print(f"\n⚠️ 未检测到完整模型，开始训练...")
    
    training_results = trainer.train_all_windows(
        model_class=HybridGraphModel,
        model_config=model_config,
        save_dir=str(save_dir),
        val_ratio=VAL_RATIO,
        incremental=False,
        dynamic_adj=False,
        adj_config=adj_config,
        use_cross_sectional=True
    )
    
    print("\n✅ 滚动窗口训练完成 (研报 Baseline)")

# =============================================================================
# 3. 保存训练结果到全局变量
# =============================================================================
# 同时保存为 results（兼容旧代码）
results = training_results

print(f"\n💡 提示: 训练结果已保存到 'training_results' 变量")
print(f"   下次运行此单元格将自动跳过训练，直接恢复模型")

2025-12-01 13:45:07 - DataManager - INFO -   自动提取全局股票池: 800 只
INFO:DataManager:  自动提取全局股票池: 800 只
2025-12-01 13:45:07 - DataManager - INFO - 
INFO:DataManager:
2025-12-01 13:45:07 - DataManager - INFO - 🔄 创建滚动窗口训练器
INFO:DataManager:🔄 创建滚动窗口训练器
2025-12-01 13:45:07 - DataManager - INFO - ================================================================================
INFO:DataManager:================================================================================
2025-12-01 13:45:07 - DataManager - INFO -   全局股票池: 800 只
INFO:DataManager:  全局股票池: 800 只
2025-12-01 13:45:07 - DataManager - INFO - ================================================================================
INFO:DataManager:================================================================================
2025-12-01 13:45:07 - DataManager - INFO - 🔄 初始化滚动窗口训练器
INFO:DataManager:🔄 初始化滚动窗口训练器
2025-12-01 13:45:07 - DataManager - INFO - ================================================================================
INFO:DataMan


🔄 步骤 3: 滚动窗口训练 (研报 Baseline)

📌 研报 Baseline 配置:
   多因子输出维度 F = 8
   静态邻接矩阵: output/industry_adj_matrix.pt

✅ 滚动窗口训练完成 (研报 Baseline)


In [ ]:
# =============================================================================
# 🚀 步骤 4: 多因子回测
# =============================================================================
# 
# 简化后的回测流程：
#   - 模型 IO 参数：自动从 dm.config 提取（stock_col, time_col, label_col）
#   - 回测参数：在此单元格集中配置（因子处理、组合设置、成本设置）
# =============================================================================

import sys
sys.path.insert(0, '/home/u2025210237/jupyterlab')

import importlib
import quantclassic.backtest
importlib.reload(quantclassic.backtest)

from quantclassic.backtest import BacktestConfig, MultiFactorBacktest
from pathlib import Path

print("=" * 80)
print("🚀 多因子回测系统")
print("=" * 80)

# =============================================================================
# 1. 模型 IO 参数（自动从 dm.config 提取，无需修改）
# =============================================================================
model_io_params = dict(
    stock_col=dm.config.stock_col,
    time_col=dm.config.time_col,
    label_col=dm.config.label_col,
)

print(f"\n【模型 IO 参数】(自动提取)")
print(f"   stock_col: {model_io_params['stock_col']}")
print(f"   time_col:  {model_io_params['time_col']}")
print(f"   label_col: {model_io_params['label_col']}")

# =============================================================================
# 2. 回测参数配置（可根据需要修改）
# =============================================================================
backtest_options = dict(
    # ===== 输出目录 =====
    output_dir=str(Path(data_config.base_dir) / 'backtest_multifactor'),
    
    # ===== 因子处理 =====
    winsorize_method='quantile',
    winsorize_quantiles=(0.025, 0.975),
    standardize_method='zscore',
    fillna_method='median',
    
    # ===== 组合设置 =====
    n_groups=10,                    # 分组数量
    rebalance_freq='biweekly',      # 调仓频率: daily, weekly, biweekly, monthly
    weight_method='equal',          # 权重方法: equal, value_weighted
    long_ratio=0.2,                 # 做多比例
    short_ratio=0.2,                # 做空比例
    
    # ===== 成本设置 =====
    consider_cost=True,
    commission_rate=0.0003,         # 佣金率
    stamp_tax_rate=0.001,           # 印花税
    slippage_rate=0.001,            # 滑点
    
    # ===== 报告设置 =====
    save_plots=True,
    generate_excel=True,
    report_title='多因子模型回测报告',
)

print(f"\n【回测参数】(可修改)")
print(f"   输出目录: {backtest_options['output_dir']}")
print(f"   分组数量: {backtest_options['n_groups']}")
print(f"   调仓频率: {backtest_options['rebalance_freq']}")
print(f"   多空比例: 多{backtest_options['long_ratio']:.0%} / 空{backtest_options['short_ratio']:.0%}")
print(f"   考虑成本: {backtest_options['consider_cost']}")

# =============================================================================
# 3. 获取预测结果
# =============================================================================
print(f"\n【获取预测结果】")

# 检查 training_results 是否存在
if 'training_results' not in dir() or training_results is None:
    raise ValueError(
        "❌ 未找到 training_results！\n"
        "   请先运行 Cell 9（训练/恢复单元格）"
    )

rolling_predictions = trainer.predict_all_windows(training_results)
print(f"   数据形状: {rolling_predictions.shape}")
print(f"   时间范围: {rolling_predictions[model_io_params['time_col']].min()} ~ "
      f"{rolling_predictions[model_io_params['time_col']].max()}")

# 检测因子列
factor_cols = [c for c in rolling_predictions.columns if c.startswith('pred_factor_')]
print(f"   检测到 {len(factor_cols)} 个因子列")

# =============================================================================
# 4. 运行回测
# =============================================================================
print(f"\n【运行回测】")

backtest_config = BacktestConfig(**backtest_options)
backtest = MultiFactorBacktest(backtest_config)

backtest_results = backtest.run(
    rolling_predictions,
    **model_io_params,
    ensemble_method='ic_weighted',  # 集成方法: mean, ic_weighted, best
    save_results=True,
)

# =============================================================================
# 5. 结果汇总
# =============================================================================
print(f"\n【回测结果汇总】")

# IC 统计
ic_stats = backtest_results['ic_stats']
print(f"\n📊 IC 统计:")
print(f"   IC 均值:  {ic_stats['ic_mean']:+.4f}")
print(f"   ICIR:     {ic_stats['icir']:.4f}")
print(f"   IC 胜率:  {ic_stats['ic_win_rate']:.2%}")

# 绩效指标
metrics = backtest_results['metrics']
if 'long_short' in metrics:
    ls = metrics['long_short']
    print(f"\n📈 多空组合绩效:")
    print(f"   年化收益: {ls['annual_return']:+.2%}")
    print(f"   年化波动: {ls['annual_volatility']:.2%}")
    print(f"   夏普比率: {ls['sharpe_ratio']:.4f}")
    print(f"   最大回撤: {ls['max_drawdown']:.2%}")

# 因子权重
weights = backtest_results.get('ensemble_weights', {})
if weights:
    print(f"\n⚖️ 因子权重 (Top 5):")
    for name, w in sorted(weights.items(), key=lambda x: -x[1])[:5]:
        print(f"   {name}: {w:.4f}")

print("\n" + "=" * 80)
print(f"✅ 回测完成！结果保存在: {backtest_options['output_dir']}")
print("=" * 80)

2025-12-01 13:50:34 - DataManager - INFO - 
INFO:DataManager:
2025-12-01 13:50:34 - DataManager - INFO - 🔮 开始滚动窗口预测
INFO:DataManager:🔮 开始滚动窗口预测
2025-12-01 13:50:34 - DataManager - INFO - ================================================================================
INFO:DataManager:================================================================================
2025-12-01 13:50:34 - DataManager - INFO -   预测窗口 1/5...
INFO:DataManager:  预测窗口 1/5...
INFO:DataManager:
2025-12-01 13:50:34 - DataManager - INFO - 🔮 开始滚动窗口预测
INFO:DataManager:🔮 开始滚动窗口预测
2025-12-01 13:50:34 - DataManager - INFO - ================================================================================
INFO:DataManager:================================================================================
2025-12-01 13:50:34 - DataManager - INFO -   预测窗口 1/5...
INFO:DataManager:  预测窗口 1/5...


🚀 多因子回测系统 - 一键运行

【0】检查训练结果
  ✓ 使用内存中的 training_results
  训练窗口数: 5

【1】获取预测结果


2025-12-01 13:51:11 - DataManager - INFO -     预测样本: 175,114
INFO:DataManager:    预测样本: 175,114
2025-12-01 13:51:11 - DataManager - INFO -   预测窗口 2/5...
INFO:DataManager:  预测窗口 2/5...
INFO:DataManager:    预测样本: 175,114
2025-12-01 13:51:11 - DataManager - INFO -   预测窗口 2/5...
INFO:DataManager:  预测窗口 2/5...
2025-12-01 13:51:50 - DataManager - INFO -     预测样本: 185,636
INFO:DataManager:    预测样本: 185,636
2025-12-01 13:51:50 - DataManager - INFO -   预测窗口 3/5...
INFO:DataManager:  预测窗口 3/5...
2025-12-01 13:51:50 - DataManager - INFO -     预测样本: 185,636
INFO:DataManager:    预测样本: 185,636
2025-12-01 13:51:50 - DataManager - INFO -   预测窗口 3/5...
INFO:DataManager:  预测窗口 3/5...
2025-12-01 13:52:27 - DataManager - INFO -     预测样本: 193,755
INFO:DataManager:    预测样本: 193,755
2025-12-01 13:52:27 - DataManager - INFO -   预测窗口 4/5...
INFO:DataManager:  预测窗口 4/5...
2025-12-01 13:52:27 - DataManager - INFO -     预测样本: 193,755
INFO:DataManager:    预测样本: 193,755
2025-12-01 13:52:27 - DataManager - INFO -   

  数据形状: (839020, 5)
  时间范围: 2020-05-20 00:00:00 ~ 2024-12-17 00:00:00
  检测到 0 个因子列

【2】创建回测配置（从 data_config 统一）
  输出目录: output/backtest_multifactor
  分组数量: 10
  调仓频率: biweekly
  多空比例: 多20% / 空20%

【3】运行多因子回测


13:53:24 - INFO - 
【3/6】IC 分析
INFO:quantclassic.Factorsystem.multi_factor_backtest:
【3/6】IC 分析
INFO:quantclassic.Factorsystem.multi_factor_backtest:
【3/6】IC 分析
13:53:26 - INFO - 
  📊 IC 统计:
INFO:quantclassic.Factorsystem.multi_factor_backtest:
  📊 IC 统计:
13:53:26 - INFO -      IC 均值:   +0.0245
INFO:quantclassic.Factorsystem.multi_factor_backtest:     IC 均值:   +0.0245
13:53:26 - INFO -      IC 标准差: 0.1552
INFO:quantclassic.Factorsystem.multi_factor_backtest:     IC 标准差: 0.1552
13:53:26 - INFO -      ICIR:      0.1575
INFO:quantclassic.Factorsystem.multi_factor_backtest:     ICIR:      0.1575
13:53:26 - INFO -      IC 胜率:   55.71%
INFO:quantclassic.Factorsystem.multi_factor_backtest:     IC 胜率:   55.71%
13:53:26 - INFO -      t 统计量:  5.2559
INFO:quantclassic.Factorsystem.multi_factor_backtest:     t 统计量:  5.2559
13:53:26 - INFO - 
【4/6】构建多空组合
INFO:quantclassic.Factorsystem.multi_factor_backtest:
【4/6】构建多空组合
13:53:26 - INFO - 
  📊 IC 统计:
INFO:quantclassic.Factorsystem.multi_factor_backtes


【4】回测结果汇总

📊 IC 统计:
   IC 均值:  +0.0245
   ICIR:     0.1575
   IC 胜率:  55.71%

📈 多空组合绩效:
   年化收益: +28.01%
   年化波动: 17.11%
   夏普比率: 1.4613
   最大回撤: -17.33%

✅ 回测完成！结果保存在: output/backtest_multifactor

💡 提示：回测结果已保存到 'backtest_results' 变量，可重复运行此单元格


In [ ]:
# =============================================================================
# 🔬 可选：对比不同集成方法
# =============================================================================
# 
# 这个单元格会自动对比 3 种集成方法：
#   - mean:        简单平均
#   - ic_weighted: IC 加权平均
#   - best:        选择最佳单因子
#
# 运行后可以看到哪种方法效果最好
#
# 🔴 修复：
#   - 确保 backtest 和 backtest_config 存在
#   - 如果不存在则自动创建（支持独立运行此单元格）
# =============================================================================

import sys
sys.path.insert(0, '/home/u2025210237/jupyterlab')

from pathlib import Path
import pandas as pd

print("=" * 80)
print("🔬 多集成方法对比")
print("=" * 80)

# =============================================================================
# 0. 检查必要变量
# =============================================================================

# 检查 rolling_predictions 是否存在
if 'rolling_predictions' not in dir():
    raise ValueError(
        "❌ 未找到 rolling_predictions！\n"
        "   请先运行上一个单元格（一键回测）生成预测结果"
    )

# 检查 backtest_config 是否存在，不存在则创建
if 'backtest_config' not in dir():
    print("  ⚠️ backtest_config 不存在，正在创建...")
    from quantclassic.backtest import BacktestConfig
    
    backtest_preset = dict(
        output_dir=str(Path(data_config.base_dir) / 'backtest_multifactor'),
        winsorize_method='quantile',
        winsorize_quantiles=getattr(data_config, 'winsorize_quantiles', (0.025, 0.975)),
        standardize_method='zscore',
        fillna_method=getattr(data_config, 'fillna_method', 'median'),
        n_groups=getattr(data_config, 'n_groups', 10),
        rebalance_freq=getattr(data_config, 'rebalance_freq', 'biweekly'),
        weight_method='equal',
        long_ratio=getattr(data_config, 'long_ratio', 0.2),
        short_ratio=getattr(data_config, 'short_ratio', 0.2),
        consider_cost=True,
        commission_rate=getattr(data_config, 'commission_rate', 0.0003),
        stamp_tax_rate=getattr(data_config, 'stamp_tax_rate', 0.001),
        slippage_rate=getattr(data_config, 'slippage_rate', 0.001),
        save_plots=True,
        generate_excel=True,
        report_title="多因子模型回测报告"
    )
    backtest_config = BacktestConfig(**backtest_preset)
    print("  ✓ backtest_config 已创建")

# 检查 backtest 是否存在，不存在则创建
if 'backtest' not in dir():
    print("  ⚠️ backtest 不存在，正在创建...")
    from quantclassic.backtest import MultiFactorBacktest
    backtest = MultiFactorBacktest(backtest_config)
    print("  ✓ backtest 已创建")

print("  ✓ 所有依赖检查通过")

# =============================================================================
# 1. 对比多种集成方法
# =============================================================================
print("\n【1】运行多集成方法对比")

comparison_results = backtest.run_multi_ensemble(
    rolling_predictions,
    stock_col=dm.config.stock_col,
    time_col=dm.config.time_col,
    label_col=dm.config.label_col,
    methods=['mean', 'ic_weighted', 'best']
)

# =============================================================================
# 2. 汇总对比表格
# =============================================================================
print("\n【2】汇总结果")

comparison_data = []
for method, res in comparison_results.items():
    method_ic_stats = res.get('ic_stats', {})
    method_metrics = res.get('metrics', {})
    ls = method_metrics.get('long_short', {})
    
    comparison_data.append({
        '集成方法': method,
        'IC均值': method_ic_stats.get('ic_mean', 0),
        'ICIR': method_ic_stats.get('icir', 0),
        'IC胜率': method_ic_stats.get('ic_win_rate', 0),
        '年化收益': ls.get('annual_return', 0),
        '夏普比率': ls.get('sharpe_ratio', 0),
        '最大回撤': ls.get('max_drawdown', 0)
    })

comparison_df = pd.DataFrame(comparison_data)

# 格式化显示
print("\n📊 集成方法效果对比:")
print(comparison_df.to_string(index=False, formatters={
    'IC均值': '{:+.4f}'.format,
    'ICIR': '{:.4f}'.format,
    'IC胜率': '{:.2%}'.format,
    '年化收益': '{:+.2%}'.format,
    '夏普比率': '{:.4f}'.format,
    '最大回撤': '{:.2%}'.format
}))

# 找出最佳方法
best_method = comparison_df.loc[comparison_df['ICIR'].idxmax(), '集成方法']
print(f"\n🏆 推荐使用: {best_method} (ICIR最高)")

print("\n" + "=" * 80)
print("✅ 对比完成！各方法结果保存在对应文件夹")
print("=" * 80)